# 1、**模型**

## **1.1 论文提出的模型**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')
!mkdir -p drive1
!google-drive-ocamlfuse drive1 # 此时colab中出现drive的文件夹，里面就是你的google drive的根目录文件

Mounted at /content/gdrive/
/bin/bash: google-drive-ocamlfuse: command not found


### **1.1.1 模块定义**

In [15]:
import os
import sys
#reload(sys)
import imp
imp.reload(sys)
import importlib
importlib.reload(sys)
#import cPickle as pkl
import _pickle as pkl
import _pickle as cPickle
import pandas as pd
import numpy as np
import argparse
import matplotlib
import time, datetime
import re
matplotlib.use('Agg')
import matplotlib.pyplot as plt
seed = 1234
np.random.seed(seed)

from random import shuffle
from collections import Counter
from sklearn import ensemble
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import Normalizer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, Flatten, Dropout, concatenate, add
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import optimizers
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

from tensorflow.keras.layers import Dense, Lambda, Dot, Activation, Concatenate, Layer, Embedding, LSTM, Dense,Attention
from keras import initializers
from keras.layers import Input, Dense, merge
from keras.models import *
#np.random.seed(1337)  # for reproducibility
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
#from tsne import tsne
from sklearn.manifold import TSNE
from gensim.models import word2vec
from gensim.models import Word2Vec, KeyedVectors
from gensim.test.utils import common_texts, get_tmpfile

parser = argparse.ArgumentParser(description='embedconv training') #建立解析对象
parser.add_argument('-batchsize', dest='batchsize', type=int, default=128, help='size of one batch')#超参数
parser.add_argument('-init', dest='init', action='store_true', default=True, help='initialize vector')
parser.add_argument('-noinit', dest='init', action='store_false', help='no initialize')
parser.add_argument('-trainable', dest='trainable', action='store_true', default=True, help='embedding vectors trainable')
parser.add_argument('-notrainable', dest='trainable', action='store_false', help='not trainable')
parser.add_argument('-transform', dest='transform', action='store_true', default=True, help='transformation of the cost')
parser.add_argument('-test', dest='test', action='store_true', default=False, help='only test step')
parser.add_argument('-filter', dest='filter', action='store_true', default=True, help='filter rare codes')
parser.add_argument('-isdays', dest='isdays', action='store_true', default=False, help='prediction of length of stay')
parser.add_argument('-dropout', dest='dpt', type=float, default=0.01, help='drop out rate')#超参数
parser.add_argument('-filtersize', dest='fz', type=int, default=3, help='filter region size')
parser.add_argument('-filternumber', dest='fn', type=int, default=100, help='filter numbers')#
parser.add_argument('-lr', dest='lr', type=float, default=0.001, help='learning rate' )#超参数
parser.add_argument('-maxlen', dest='maxlen', type=int, default=17, help='max sequence length')
parser.add_argument('-dim', dest='dim', type=int, default=900, help='embedding vector length')#
parser.add_argument('-window', dest='window', type=int, default=10, help='word2vec window size')
args = parser.parse_args(args=[])

dropout = args.dpt
filter_size = args.fz
filter_number = args.fn
lr = args.lr
embedding_vector_length = args.dim
print('embedding vector length %d'%(embedding_vector_length))

if args.test:
    MID = 29
    SID = 48
else:
    i = datetime.datetime.now()
    MID = i.minute
    SID = i.second
f = '/content/gdrive/MyDrive/15684_new_2.csv'
p = r'^[A-Z]'
pattern = re.compile(p) #将一个正则表达式编译成 Pattern 对象，可以利用 pattern 的一系列方法对文本进行匹配查找

def DataClean ():
    print('开始步骤1:数据的清洗和导入')
    data = pd.read_csv('/content/gdrive/MyDrive/15684_new_2.csv', encoding='utf-8')
    data = data[['target','Hospital_days', #预测目标值
                'Historical_days','Age','Hospital_times','Historical_times','time_interval','Date','Year','Month','Week','Historical_diagnoses',#患者特征
                'Diagnostic_code6']] #疾病特征
    data = data.dropna(subset=['Historical_days','Age','Hospital_times','Diagnostic_code6',]) #该函数主要用于滤除缺失数据
    cPickle.dump(data, open('/content/gdrive/MyDrive/dataclean.df', 'wb')) ##把data写入到后面链接文件中
    return data

def ToRawList(data):
    print('开始步骤2:形成原始单词清单')
    n_samples = len(data.index)
    # demographics, P27=days
    demographics = np.zeros((n_samples, 10))
    demographics[:, 0:1] = data[['Historical_days']].values 
    demographics[:, 1:2] = data[['Age']].values 
    demographics[:, 2:3] = data[['Hospital_times']].values
    demographics[:, 3:4] = data[['Historical_times']].values
    demographics[:, 4:5] = data[['Date']].values
    demographics[:, 5:6] = data[['Year']].values
    demographics[:, 6:7] = data[['Month']].values
    demographics[:, 7:8] = data[['Week']].values
    demographics[:, 8:9] = data[['Historical_diagnoses']].values
    demographics[:, 9:10] = data[['time_interval']].values
    #diseases codes
    disease = data[['Diagnostic_code6']]
    disease = disease.fillna('')
    disease = disease.values
    disease = [[str(code).strip() for code in item if code != ''] for item in disease]
    #print(disease[:5])
    main_dis = data[['Diagnostic_code6']]
    main_dis = main_dis.fillna('')
    main_dis = main_dis.values
    l_disease = [len(item) for item in disease]
    l_disease = np.array(l_disease)
    C_disease = Counter(l_disease)
    #print("l_disease的最大长度:{}, 最小长度:{}".format(np.max(l_disease),np.min(l_disease))) 
    #print("l_disease的the 25% quarter:{}, the 75% quarter:{}, the mean:{}(+-){}".format(np.percentile(l_disease,25),np.percentile(l_disease,75),np.mean(l_disease),np.std(l_disease))) 
    l_disease = np.array(l_disease)

    seqs = data[['Historical_days','Age','Hospital_times','Historical_times','time_interval','Date','Year','Month','Week','Historical_diagnoses','Diagnostic_code6']]
    #seqs = data[['Diagnostic_code6']]
    seqs = seqs.fillna('')
    seqs = seqs.values
    seqs = [['#'.join(str(code).strip().split(' ')) for code in item if code != ''] for item in seqs]#replace the space with '#''
    seqs = [[str('0'+code) if len(code.split('.')[0])==1 else code for code in item] for item in seqs] #replace '3.90034' with the '03.90034'
    #print(seqs[:5])

    cost = data[['target']].values
    cost = np.asarray(cost, dtype=np.float32)
    days = data[['Hospital_days']].values
    days = np.asarray(days, dtype=np.float32)

    n_samples = len(seqs) #返回疾病编码的个数，包括相同值，即12548/15684
    #print(f,'#samples(seqs的长度):%d'%(n_samples)) # %d：以十进制形式输出带符号整数(正数不输出符号)

    main_dis = [[str(code) for code in item if code != ''] for item in main_dis] #输出每个单词
    C_maincodes = Counter([code for seq in main_dis for code in seq]) #计数，求每个单词出现的次数
    main_code = C_maincodes.keys() #形成单词字典
    n_dim = len(main_code) #共有242个单词
    #print("len(main_code):{}".format(n_dim))

    code2id = dict(zip(main_code, range(n_dim))) #zip：打包为元组的列表，相对于编号
    maincodemat = np.zeros((n_samples, n_dim), dtype=np.float32) #形成12548*242的0矩阵
    for i in range(n_samples):
        for code in main_dis[i]:
            if code in code2id:
                index = code2id[code]
                maincodemat[i,index] += 1
    #print("maincodemat.shape:{}".format(maincodemat.shape)) #形成单词矩阵maincodemat，出现，则该行为1
    Data = (seqs, cost, days, demographics,disease)
    pkl.dump(Data, open('/content/gdrive/MyDrive/Data.df','wb'))
    return seqs, cost, days, demographics, disease, main_dis

#步骤3:给单词编号
def token_to_index(seqs):
    print('开始步骤3:给单词编号,形成单词-编号的字典')
    C_codes = Counter([code for seq in seqs for code in seq])
    code_index = {}
    for idx, item in enumerate(C_codes.keys()): #enumerate：同时列出数据、数据下标
        code_index[item] = idx + 1
    #print("the unique codes: {}".format(len(C_codes.keys())))
    return code_index   #输入code_index，输出：{'E10.901': 1,...}

def get_index_embedding(code_index={}, level=0):
    print("开始第4步")
    index_embedding = {}
    cnt = 0
    dim = 900
    window = 10    #glove文件包含各种大小的文本编码向量:50维、100维、200维、300维
    #model = KeyedVectors.load_word2vec_format(word_vector_path, binary=True)
    model = KeyedVectors.load('/content/gdrive/MyDrive/newlevel%d_word2vec_dim%d_window%d.model' %(level, dim, window))
   
    for code, index in code_index.items():  # items(): 返回可遍历的(键, 值) 元组数组
        #print(re.findall(pattern, code))
        if len(re.findall(pattern, code))== 1:
            if level == 0:
                newcode = code
            if level == 1:
                newcode = code[0:3]
            if level == 2:
                newcode = code[0:5]
            if level == 3:
                newcode = code[0:6]
        else:
            if level == 0:
                newcode = code
            if level == 1:
                newcode = code[0:3]
            if level == 2:
                newcode = code[0:4]
            if level == 3:
                newcode = code[0:5]

        if newcode in model:
            index_embedding[index] = model[newcode]
        else:
            cnt = cnt + 1
            index_embedding[index] = np.random.uniform(-0.25,0.25,embedding_vector_length)
    return index_embedding #即619个疾病代码，每个代码映射成了100维的向量

def get_trained_embedding(index_embedding=None):
    print("开始get_trained_embedding")
    index_sorted = sorted(index_embedding.items()) # sorted by index starting from 1 ;items(): 返回可遍历的(键, 值) 元组数组
    trained_embedding = [t[1] for t in index_sorted] #len(trained_embedding):424 type:<class 'list'>
    embedding_vector_length = args.dim
    zeros = np.random.uniform(-0.25,0.25,embedding_vector_length) #一维的数组  zeros.shape:(424,)
    trained_embedding = np.vstack((zeros, trained_embedding)) #它是垂直（按照行顺序）的把数组给堆叠起来。
    trained_embedding = np.array(trained_embedding) #trained_embedding.shape:(101, 424);trained_embedding.维度:2
    return trained_embedding

#步骤5:单尺度编码 E10.901 C12.783
def embedding_encoder(idseqs, index_embedding):
    print('开始步骤51/3:单尺度编码')
    n_samples = len(idseqs)
    dim = args.dim
    mat = np.zeros((n_samples, dim), np.float32)
    for i in range(n_samples):
        for codeid in idseqs[i]:
            if codeid in index_embedding:
                code_embedding = np.array(index_embedding[codeid])
            else:
                code_embedding = np.random.uniform(-0.25, 0.25, dim)
            mat[i] += code_embedding
    mat = np.array(mat)
    return mat

#多尺度编码 E10.901 C12.783     E10.90 C12.78  E10.9      6-5-4-3
def multichannel_embedding_encoder(idseqs, index_embedding,index_embedding1,index_embedding2,index_embedding3):
    print('开始步骤52/3:开始多尺度编码...')
    n_samples = len(idseqs)
    dim = args.dim
    mat = np.zeros((n_samples, dim), np.float32)
    for i in range(n_samples):
        for codeid in idseqs[i]:
            code_embedding = np.asarray([0]*dim, np.float32)
            if codeid in index_embedding:
                code_embedding += np.array(index_embedding[codeid])
            if codeid in index_embedding1:
                code_embedding += np.array(index_embedding1[codeid])
            if codeid in index_embedding2:
                code_embedding += np.array(index_embedding2[codeid])
            if codeid in index_embedding3:
                code_embedding += np.array(index_embedding3[codeid])
            mat[i] += code_embedding
    mat = np.array(mat) 
    return mat

#独热码
def one_hot_encoder(seqsind, nb_words):
    print('开始步骤53/3:开始独热编码one_hot encoding...')
    n_samples = len(seqsind)
    n_dim = nb_words
    mat = np.zeros((n_samples, n_dim), dtype=np.float32)
    for i in range(n_samples):
        for codeid in seqsind[i]:
            if codeid !=0:
                index = codeid - 1 # for the seqid starts from 1
                mat[i,index] = mat[i,index] + 1
    return mat

#步骤6:
def svd(seqsind, dim=args.dim): #600 奇异值分解(SVD)
    print("开始61/3:开始svd")
    mat = one_hot_encoder(seqsind, nb_words)
    #print("mat.shape:{}".format(mat.shape))
    from sklearn.decomposition import TruncatedSVD
    svd = TruncatedSVD(n_components=dim, random_state=42)
    print('SVD......') 
    svd.fit(mat)
    res = svd.transform(mat)
    print('SVD done!') 
    return res

def glove(level, dim=args.dim, window=10): 
    print("开始62/3:开始glove")
    vectors = '/content/gdrive/MyDrive/weighted1_vectors%d'%(level)
    model = KeyedVectors.load_word2vec_format(vectors, binary=False)
    name = 'newlevel%d_weighted1_glove_dim%d_window%d.model'%(level, dim, window)
    model.save('/content/gdrive/MyDrive/cnn_model/'+name)
    return model

def plot_embedding(data, label, title):
    x_min, x_max = np.min(data, 0), np.max(data, 0)
    data = (data - x_min) / (x_max - x_min)
 
    fig = plt.figure()
    ax = plt.subplot(111)
    for i in range(data.shape[0]):
        plt.text(data[i, 0], data[i, 1], str(label[i]),
                 color=plt.cm.Set1(label[i]),
                 fontdict={'weight': 'bold', 'size': 9})
    plt.xticks([])
    plt.yticks([])
    plt.title(title)
    return fig

def word2vec_model(disease, level, window=args.window, dim=args.dim, visualize=True): #seqs
    print("开始63/3:开始word2vec")
    n_samples = len(disease)
    sentences = word2vec.LineSentence('/content/gdrive/MyDrive/15684diagnosis.txt')
    
    sentences = []
    for sentence in disease:
        shuffle(sentence)
        sentences.append(sentence)
    
    model = word2vec.Word2Vec(sentences, size=dim,sg=1,min_count=1,window=window)
    model.wv.save_word2vec_format("/content/gdrive/MyDrive/word2vec.vector", binary=True)
    model.save('/content/gdrive/MyDrive/newlevel%d_word2vec_dim%d_window%d.model' %(level, dim, window))
    print("model:",model) 
    if visualize:
        codes_vocabulary = Counter([code for seq in disease for code in seq]).keys()
        #print("codes_vocabulary :",codes_vocabulary ) #codes_vocabulary : dict_keys(['E10.901', 'O24.901', 返回值返回一个字典所有的键
        X = np.vstack([model[item][np.newaxis,:] for item in codes_vocabulary])
        #print("X:",X,"X.shape:",X.shape) #(242,900)
        Y = TSNE(n_components=2, init='pca', verbose=1).fit_transform(X)
        #print("Y:",Y,"Y.shape:",Y.shape) #(242,2)
        codes_vocabulary11=list(codes_vocabulary)
        #print("codes_vocabulary11:",codes_vocabulary11) #codes_vocabulary11: ['E10.901', 'O24.901', 'I67.801', 

        pkl.dump(codes_vocabulary11, open('/content/gdrive/MyDrive/codes_vocabulary.pkl', 'wb')) #pickle.dump() 直接把对象序列化后，将对象obj保存到文件file(这里的file是文件句柄) 中去。
        pkl.dump(X, open('/content/gdrive/MyDrive/word2vec_vector.pkl', 'wb'))
        pkl.dump(Y, open('/content/gdrive/MyDrive/word2vec_2d_vector.pkl', 'wb'))
        import pickle
        f=open(r'/content/gdrive/MyDrive/word2vec_2d_vector.pkl', 'rb')
        Y = pickle.load(f)
        fig, ax = plt.subplots()
        aa=[]
        for a in codes_vocabulary11:aa.append(a[0])
        label = np.array(aa)
        #print("label:",label,"label.dtype:",label.shape) #(242,)
        #print(label[0],label[1])
        tsne = TSNE(n_components=2, init='pca', verbose=1)
        groups = pd.DataFrame(Y, columns=['x', 'y']).assign(category=label).groupby('category')
        for name, points in groups:
          plt.title('Visualization of medical concept vectors by t-SNE', size=10)
          ax.scatter(points.x, points.y, label=name, marker='o')
          ax.legend(loc=1,fontsize=5,frameon=True)
        for i in range(len(codes_vocabulary11)):
          ax.text(Y[i,0], Y[i,1], codes_vocabulary11[i], fontsize=4, 
          color = "g", style = 'normal', weight = "light", verticalalignment='center', 
          horizontalalignment='right',rotation=0) #给散点加标签
        fig.savefig('/content/gdrive/MyDrive/%s_tsne.pdf'%(level))
    #return model


#步骤7:加载数据
def load_data(X,demographics, y, onehot_mat):
    print('开始步骤7:划分训练集、验证集、测试集...')
    print("y:",y)
    #y=np.log(y)
    indices = np.arange(n_seqs)
    np.random.shuffle(indices) #打乱
    X = X[indices]
    demographics = demographics[indices]
    y = y[indices]
    #onehot_mat = onehot_mat[indices]

    n_tr = int(n_seqs * 0.85)
    n_va = int(n_seqs * 0.05)
    n_te = n_seqs - n_tr - n_va
    X_train = X[:n_tr]
    demographics_train = demographics[:n_tr]
    y_train = y[:n_tr]
    #onehot_mat_train = onehot_mat[:n_tr]

    X_valid = X[n_tr:n_tr+n_va]
    demographics_valid = demographics[n_tr:n_tr+n_va]
    y_valid = y[n_tr:n_tr+n_va]
    #onehot_mat_valid = onehot_mat[n_tr:n_tr+n_va]

    X_test = X[-n_te:]
    demographics_test = demographics[-n_te:]
    y_test = y[-n_te:]
    #onehot_mat_test = onehot_mat[-n_te:]

    #print np.max(y_test),np.min(y_test)
    return X_train, X_test, X_valid, y_train, y_test, y_valid, demographics_train, demographics_test, demographics_valid#, onehot_mat_train,onehot_mat_test,onehot_mat_valid


def filter_test(X_test, index_code, threshold=2):
    print('开始步骤8:选择罕见集样本用于测试集...，这一步其实是将所有数据15684都用于了测试集')
    comm_inds = []
    for ind, seq in enumerate(X_test):
        cnt = 0
        for index in seq:
            if index in index_code:
                code = index_code[index]
            else:
                code = 'none'
        if cnt < threshold:
            comm_inds.append(int(ind))
    print('comm_inds:%d'%(len(comm_inds)))
    comm_inds = np.asarray(comm_inds)
    return  comm_inds

#步骤8:预测、可视化
def cross_validation():
    print("开始5折交叉验证")
    estimator = KerasRegressor(build_fn=cnn_model, epochs=100, batch_size=args.batchsize, verbose=0)
    kfold = KFold(n_splits=5, random_state=seed)
    res = cross_val_score(estimator, X, y, cv=kfold)
    print( 'cross validation for cnn model ......')
    print('cnn_model: MSE %.2f (+-)%.2f'%(res.mean(), res.std())) 
#cross_validation()

def evaluation(y_test, y_pred):
    print('y_test:{}'.format(y_test.shape))
    print('y_pred:{}'.format(y_pred.shape))
    print(y_pred.dtype) #float32
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print('MSE:{}, RMSE:{}, MAE:{}, R2:{}'.format(mse, rmse, mae,r2)) 
    return r2, rmse

def daysplots(y_test, y_pred, r2, rmse, name):
    print("开始步骤91/2:预测可视化-住院天数")
    fig, ax = plt.subplots()
    #ax.set_xlim(0, 35)
    #ax.set_ylim(0, 35)
    #ax.plot(y_test.squeeze(), y_test.squeeze(), s=10, marker='.',c='r')
    ax.scatter(y_test.squeeze(), y_pred, s=10,marker='.', c='b')
    ax.text(28,30,'R2:{:.4f}'.format(r2))
    ax.text(28,32,'RMSE:{:.2f}'.format(rmse))
    ax.set_xlabel('True length of stay (days)')
    ax.set_ylabel('predicted length of stay (days)')
    fig.savefig('/content/gdrive/MyDrive/%s_days.pdf'%(name))

def costplots(y_test, y_pred, r2, rmse, name):
    print("开始步骤92/2:预测可视化-医疗费用")
    fig, ax = plt.subplots()
    #ax.set_xlim(0, 12000)
    #ax.set_ylim(0, 12000)
    ax.scatter(y_test.squeeze(), y_pred, s=10,marker='o',c='b') #c='b'
   # ax.text('R2:{:.4f}'.format(r2)) #8000,10000,
    #ax.text('RMSE:{:.2f}'.format(rmse)) #8000,11000,
    ax.set_xlabel('Series')
    ax.set_ylabel('Medical cost')
    fig.savefig('/content/gdrive/MyDrive/%s_cost.png'%(name),dpi=500,bbox_inches = 'tight')

    #画散点图
    plt.figure()#figsize=(15,5)
    plt.plot(y_test,'rs',label='True value')
    plt.plot(y_pred,'go',label='Predict value')
    #plt.title('Predicted and true values of medical costs',fontsize=13)
    plt.xticks(fontsize=13)
    plt.yticks(fontsize=13)
    plt.xlabel('Series',fontsize=13)
    plt.ylabel('Medical Cost',fontsize=13)
    plt.legend(loc=1,fontsize=10,frameon=True)
    plt.savefig('/content/gdrive/MyDrive/%s_散点图.png'%(name),dpi=500,bbox_inches = 'tight')

    fig, ax = plt.subplots()
    plt.plot(y_test, linewidth = 1,linestyle='-',label='True Value') #marker='*'
    plt.plot(y_pred, linewidth = 1,linestyle='-',label='Predicted Value') #marker='^',
    #ax.text('R2:{:.4f}'.format(r2)) #8000,10000,
    #ax.text('RMSE:{:.2f}'.format(rmse)) #8000,11000,
    plt.xlabel('Series',fontsize=13)
    plt.ylabel('Medical Cost',fontsize=13)
    plt.xlabel('Series',fontsize=13)
    plt.ylabel('Medical Cost',fontsize=13)
    fig.savefig('/content/gdrive/MyDrive/%s_曲线图.pdf'%(name))

def extract_patientvec(model,modelpath, disease,  demographics):
    model.load_weights("/content/gdrive/MyDrive/cnn_model/%s.hdf5"%(modelpath),by_name=False)
    sub_model1 = Model(inputs=model.inputs, outputs=model.get_layer('F1').output)#错误代码修改
    sub_model2 = Model(inputs=model.inputs, outputs=model.get_layer('F2').output)#错误代码修改
    sub_model3 = Model(inputs=model.inputs, outputs=model.get_layer('F3').output)#错误代码修改
    sub_model4 = Model(inputs=model.inputs, outputs=model.get_layer('F4').output)#错误代码修改
    sub_model5 = Model(inputs=model.inputs, outputs=model.get_layer('F5').output)#错误代码修改

    patientvecs3 = sub_model3.predict([disease,  demographics], verbose=1)
    modelname='sub_model3'
    print(patientvecs3.shape)
    pkl.dump(patientvecs3, open('/content/gdrive/MyDrive/cnn_model/patientvec_%s'%(modelname),'wb'))

    patientvecs4 = sub_model4.predict([disease, demographics], verbose=1)
    modelname='sub_model4'
    print(patientvecs4.shape)
    pkl.dump(patientvecs4, open('/content/gdrive/MyDrive/cnn_model/patientvec_%s'%(modelname),'wb'))

    patientvecs5 = sub_model5.predict([disease,  demographics], verbose=1)
    modelname='sub_model5'
    print(patientvecs5.shape)
    pkl.dump(patientvecs5, open('/content/gdrive/MyDrive/cnn_model/patientvec_%s'%(modelname),'wb'))


embedding vector length 900


### **1.1.2 对比模型的定义**

In [3]:
#单尺度模型
####这里这里这里！！！！！！！！！！
from tensorflow.keras import optimizers # 错误代码1的改正方案
#模型1:单尺度融合模型
def single_channel_merge_model(demgras_dim):
    print('开始模型1:单尺度融合模型')
    codes_in = Input(shape=(MAX_LEN, ), dtype='float32') #MAX_len 输入句子的最大长度17
    if args.init:
        print('initialize embedding layer with pre-training vectors')
        print('依据预训练向量 初始化嵌入层')
        print('embedding layers trainalble %s' % args.trainable) 
        embedding0level = Embedding(output_dim=embedding_vector_length,  #424
                                input_dim=max_features, 
                                input_length=MAX_LEN,
                                weights=[embedding_matrix0],
                                trainable=args.trainable)(codes_in)
    else:
        print('one hot embedding with random initialization...') 
        embedding0level = Embedding(output_dim=embedding_vector_length,
                                    input_dim=max_features,
                                    embeddings_initializer='random_uniform',
                                    input_length=MAX_LEN)(codes_in)
    conv_result = []
    for i in range(3):
        conv_layer = Conv1D(filter_number, filter_size, padding='same', activation='relu')
        conv0 = conv_layer(embedding0level)
        pooling0 = GlobalMaxPooling1D()(conv0)
        conv_result.append(pooling0)

    print('添加人口统计信息，融合')
    demgras_in = Input(shape=(demgras_dim,), dtype='float32')
    dense_demgras = Dense(3, activation='sigmoid')(demgras_in)
    conv_result.append(dense_demgras)#append the demographics information
    merge_out = concatenate(conv_result)
    dense_out = Dense(500, activation='relu')(merge_out)
    dpt = Dropout(dropout)(dense_out)
    dense_out = Dense(100, activation='relu')(dpt)
    dpt = Dropout(dropout)(dense_out)
    mode_out = Dense(1)(dpt)
    model = Model([codes_in, demgras_in], mode_out)
    #rmsprop = optimizers.RMSprop(lr=lr, rho=0.9, epsilon=10-8, decay=0.0) 出错代码1
    #rmsprop = optimizers.RMSprop(lr=lr, rho=0.9, epsilon=10-8, decay=0.0) #weight_decay的作用是正则化
    rmsprop = tf.keras.optimizers.RMSprop(lr=lr, rho=0.9, epsilon=10-8, decay=0.0)
    model.compile(loss='mean_squared_error', optimizer=rmsprop, metrics=['mae']) #MSE均方误差：真实值-预测值 然后平方之后求和平均 #MAE：平均绝对误差
    print(model.summary())
    return model

#模型2:单尺度划分模型
def single_channel_split_model(demgras_dim):
    dis_in = Input(shape=(DIS_MAX_LEN, ), dtype='float32')
    if args.init:
        print('initialize embedding layer with pre-training vectors')
        print('依据预训练向量，初始化嵌入层...')
        print('embedding layers trainalble %s' % args.trainable) 
        dis_embedding0level = Embedding(output_dim=embedding_vector_length,
                                        input_dim=max_features,
                                        input_length=DIS_MAX_LEN,
                                        weights=[embedding_matrix0],
                                        trainable=args.trainable)(dis_in)

    else:
        print('one hot embedding with random initialization...')
        print('随机初始化,独热嵌入层...')
        dis_embedding0level = Embedding(output_dim=embedding_vector_length,
                                input_dim=max_features,
                                input_length=DIS_MAX_LEN,
                                embeddings_initializer='random_uniform')(dis_in)

    conv_result = []
    for i in range(3):
        conv_layer = Conv1D(filter_number, filter_size, padding='same', activation='relu')
        conv0 = conv_layer(dis_embedding0level)
        pooling0 = GlobalMaxPooling1D()(conv0)
        conv_result.append(pooling0)

    demgras_in = Input(shape=(demgras_dim,), dtype='float32')
    dense_demgras = Dense(3, activation='sigmoid')(demgras_in)
    conv_result.append(dense_demgras)#append the demographics information
    merge_out = concatenate(conv_result)
    dense_out = Dense(1000, activation='relu')(merge_out)
    dpt = Dropout(dropout)(dense_out)
    dense_out = Dense(500, activation='relu')(dpt)
    dpt = Dropout(dropout)(dense_out)
    dense_out = Dense(100, activation='relu')(dpt)
    dpt = Dropout(dropout)(dense_out)
    mode_out = Dense(1)(dpt)
    model = Model([dis_in,  demgras_in], mode_out) #sur_in,
    rmsprop = tf.keras.optimizers.RMSprop(lr=lr, rho=0.9, epsilon=10-8, decay=0.0)
    model.compile(loss='mean_squared_error', optimizer=rmsprop, metrics=['mae'])
    print(model.summary())
    return model
#第三个模型：多尺度划分模型
def multi_channel_split_model(demgras_dim):
    dis_in = Input(shape=(DIS_MAX_LEN, ), dtype='float32')
    dis_embedding0level = Embedding(output_dim=embedding_vector_length,
                                input_dim=max_features,
                                input_length=DIS_MAX_LEN,
                                weights=[embedding_matrix0],
                                trainable=args.trainable)(dis_in)
    dis_embedding1level = Embedding(output_dim=embedding_vector_length,
                                input_dim=max_features,
                                input_length=DIS_MAX_LEN,
                                weights=[embedding_matrix1],
                                trainable=args.trainable)(dis_in)
    dis_embedding2level = Embedding(output_dim=embedding_vector_length,
                                input_dim=max_features,
                                input_length=DIS_MAX_LEN,
                                weights=[embedding_matrix2],
                                trainable=args.trainable)(dis_in)
    dis_embedding3level = Embedding(output_dim=embedding_vector_length,
                                input_dim=max_features,
                                input_length=DIS_MAX_LEN,
                                weights=[embedding_matrix3],
                                trainable=args.trainable)(dis_in)

    conv_result = []
    for i in range(3):
        channel_result = []
        conv_layer = Conv1D(filter_number, filter_size, padding='same', activation='relu')
        conv0 = conv_layer(dis_embedding0level)
        conv1 = conv_layer(dis_embedding1level)
        conv2 = conv_layer(dis_embedding2level)
        conv3 = conv_layer(dis_embedding3level)
        pooling0 = GlobalMaxPooling1D()(conv0)
        pooling1 = GlobalMaxPooling1D()(conv1)
        pooling2 = GlobalMaxPooling1D()(conv2)
        pooling3 = GlobalMaxPooling1D()(conv3)
        channel_result.append(pooling0)
        channel_result.append(pooling1)
        channel_result.append(pooling2)
        channel_result.append(pooling3)
        allchannel = add(channel_result)
        conv_result.append(allchannel)
    demgras_in = Input(shape=(demgras_dim,), dtype='float32')
    print("demgras_dim:{}".format(demgras_dim)) #demgras_dim:11
    print("demgras_in.shape:{}".format(demgras_in.shape))#(None, 11)
    dense_demgras = Dense(1000, activation='sigmoid')(demgras_in)
    conv_result.append(dense_demgras)#append the demographics information
    merge_out = concatenate(conv_result) #numpy.concatenate函数 主要作用:沿现有的某个轴对一系列数组进行拼接。
    dense_out = Dense(1000, activation='relu', name='F1')(merge_out)
    dpt = Dropout(dropout)(dense_out)
    dense_out = Dense(500, activation='relu', name='F2')(dpt) 
    dpt = Dropout(dropout)(dense_out)
    dense_out = Dense(100, activation='relu', name='F3')(dpt)
    dpt = Dropout(dropout)(dense_out)
    dense_out = Dense(50, activation='relu', name='F4')(dpt)
    dpt = Dropout(dropout)(dense_out)
    dense_out = Dense(2, activation='relu', name='F5')(dpt)
    dpt = Dropout(dropout)(dense_out)
    mode_out = Dense(1)(dpt)
    model = Model([dis_in,demgras_in], mode_out) #sur_in, 
    #rmsprop = optimizers.rmsprop(lr=lr, rho=0.9, epsilon=10-8, decay=0.0) 均方根传播(RMSProp) ;根据最近的权重梯度的平均值(例如变化的速度)来调整;这意味着该算法在线上和非平稳问题上表现良好(如:噪声)。
    Adam=tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.99, epsilon=1e-08, decay=0.0)
    model.compile(loss='mean_squared_error', optimizer=Adam, metrics=['mae']) #rmsprop
    print(model.summary())
    return model

def train_single_channel_merge_model(model, modelpath, X_train, demographics_train, y_train):
    #train_mat,valid_mat, train_y, valid_y, demographics_train, demographics_valid = train_test_split(X_train, y_train, demographics_train, test_size=0.05, random_state=seed)
    checkpointer = ModelCheckpoint(filepath="/content/gdrive/MyDrive/cnn_model/%s.hdf5"%(modelpath),verbose=1, save_best_only=True)
    earlystopper = EarlyStopping(monitor='val_loss', patience=6, verbose=1)

    print('Training the merge model....')
    model.fit([X_train, demographics_train], y_train, epochs=200, batch_size=args.batchsize, shuffle=True,
              validation_split=0.1,
              callbacks=[checkpointer,earlystopper],
              verbose=1)

def train_single_channel_split_model(model, modelpath, X1_train, demographics_train, y_train):
    #train_mat,valid_mat, train_y, valid_y, demographics_train, demographics_valid = train_test_split(X_train, y_train, demographics_train, test_size=0.05, random_state=seed)
    checkpointer = ModelCheckpoint(filepath="/content/gdrive/MyDrive/cnn_model/%s.hdf5"%(modelpath),verbose=1, save_best_only=True)
    earlystopper = EarlyStopping(monitor='val_loss', patience=6, verbose=1)

    print('Training the merge model....') 
    model.fit([X1_train,  demographics_train], y_train, epochs=200, batch_size=args.batchsize, shuffle=True,
              validation_split=0.1,
              callbacks=[checkpointer,earlystopper],
              verbose=1)

def train_multi_channel_split_model(model, modelpath, X1_train,  demographics_train, y_train):
    #train_mat,valid_mat, train_y, valid_y, demographics_train, demographics_valid = train_test_split(X_train, y_train, demographics_train, test_size=0.05, random_state=seed)
    checkpointer = ModelCheckpoint(filepath="/content/gdrive/MyDrive/cnn_model/%s.hdf5"%(modelpath),verbose=1, save_best_only=True)
    earlystopper = EarlyStopping(monitor='val_loss', patience=6, verbose=1)

    print('Training the merge model....') 
    model.fit([X1_train,  demographics_train], y_train, epochs=200, batch_size=args.batchsize, shuffle=True,
              validation_split=0.1,
              callbacks=[checkpointer,earlystopper],
              verbose=1) 

def test_single_channel_merge_model(model, modelpath,X_test,demographics_test,y_test, index):
    print('Testing model...') 
    model.load_weights("/content/gdrive/MyDrive/cnn_model/%s.hdf5"%(modelpath))
    y_pred = model.predict([X_test, demographics_test], batch_size=args.batchsize, verbose=1)
    r2,rmse = evaluation(y_test, y_pred)
    print(r2)
    name = 'SG_merge%d'%(index)
    if args.isdays:
        daysplots(y_test, y_pred, r2, rmse, name)
    else:
        costplots(y_test, y_pred, r2, rmse, name)

def test_single_channel_split_model(model, modelpath, X1_test,  demographics_test, y_test,index):
    print('Testing model...') 
    model.load_weights("/content/gdrive/MyDrive/cnn_model/%s.hdf5"%(modelpath))
    y_pred = model.predict([X1_test,  demographics_test], batch_size=args.batchsize, verbose=1)
    r2,rmse = evaluation(y_test, y_pred)
    name = 'SG_split%d'%(index)
    if args.isdays:
        daysplots(y_test, y_pred, r2, rmse, name)
    else:
        costplots(y_test, y_pred, r2, rmse, name)

def test_multi_channel_split_model(model, modelpath, X1_test,  demographics_test, y_test,index):
    print('Testing model...') 
    model.load_weights("/content/gdrive/MyDrive/cnn_model/%s.hdf5"%(modelpath))
    y_pred = model.predict([X1_test, demographics_test], batch_size=args.batchsize, verbose=1)
    print("y_pred:{}".format(y_pred.shape))
    r2,rmse = evaluation(y_test, y_pred)
    name = 'MG_split%d'%(index)
    if args.isdays:
        daysplots(y_test, y_pred, r2, rmse, name)
    else:
        costplots(y_test, y_pred, r2, rmse, name)

### **1.1.3 我们的模型**

In [4]:
#第四个模型：多尺度划分——注意力模型
# se注意力机制 现有注意力模块的另一个重要影响因素：权值生成方法。现有注意力往往采用额外的子网络生成注意力权值，比如SE的GAP+FC+ReLU+FC+Sigmoid。
def se_block(inputs, ratio=2):  # ratio代表第一个全连接层下降通道数的系数
    in_channel = inputs.shape[-1]# 获取输入特征图的通道数
    print('通道数:{}'.format(in_channel))
    x = layers.GlobalAveragePooling1D()(inputs) # 全局平均池化[h,w,c]==>[None,c] #压缩：得到当前Feature Map的全局压缩特征量
    x = layers.Reshape(target_shape=(1,1,in_channel))(x) # [None,c]==>[1,1,c]
    x = layers.Dense(in_channel//ratio)(x)  # 全连接下降通道数 # [1,1,c]==>[1,1,c/4] #激发：通过两层全连接的bottleneck结构得到Feature Map中每个通道的权值
    x = tf.nn.relu(x) # relu激活
    x = layers.Dense(in_channel)(x)  # 全连接上升通道数 # [1,1,c/4]==>[1,1,c]
    x = tf.nn.sigmoid(x) # sigmoid激活，权重归一化
    print("sigmoid激活，权重归一化:",x)
    outputs = layers.multiply([inputs, x])  # 归一化权重和原输入特征图逐通道相乘 # [h,w,c]*[1,1,c]==>[h,w,c] #激发：并将加权后的Feature Map作为下一层网络的输入。
    return outputs  
#eca注意力机制：相比于se模块，实现了适当的跨通道交互而不是像全连接层一样全通道交互。（用一维卷积替换了全连接层）
#通过执行卷积核大小为k的一维卷积来生成通道权重，其中k通过通道维度C的映射自适应地确定。
import math
def eca_block(inputs, b=1, gama=2):
    in_channel = inputs.shape[-1] # 输入特征图的通道数
    print('通道数:{}'.format(in_channel))
    kernel_size = int(abs((math.log(in_channel, 2) + b) / gama)) # 根据公式计算自适应卷积核大小
    if kernel_size % 2:              # 如果卷积核大小是偶数，就使用它
        kernel_size = kernel_size
    else:                            # 如果卷积核大小是奇数就变成偶数
        kernel_size = kernel_size + 1
    x = layers.GlobalAveragePooling1D()(inputs) # [h,w,c]==>[None,c] 全局平均池化
    x = layers.Reshape(target_shape=(in_channel, 1))(x) # [None,c]==>[c,1]
    x = layers.Conv1D(filters=1, kernel_size=kernel_size, padding='same', use_bias=False)(x) # [c,1]==>[c,1]
    x = tf.nn.sigmoid(x) # sigmoid激活
    x = layers.Reshape((1,1,in_channel))(x) # [c,1]==>[1,1,c]
    outputs = layers.multiply([inputs, x]) # 结果和输入相乘
    return outputs
##
def prelu(_x): #name=None
    """parametric ReLU activation"""
    print("_x:",_x.get_shape(),"_x.get_shape()[-1]:",_x.get_shape()[-1])
    _alpha = tf.compat.v1.get_variable("prelu", #name +
                             shape=_x.get_shape()[-1], #get_shape():得到张量（数组）的维度
                             dtype=_x.dtype,
                             initializer=tf.constant_initializer(0.001))
    pos = tf.nn.relu(_x)
    neg = _alpha * (_x - tf.abs(_x)) * 0.5

    return pos + neg


import tensorflow as tf
from keras.layers import BatchNormalization

def ATTENTION_multi_channel_split_model(demgras_dim):
    dis_in = Input(shape=(DIS_MAX_LEN, ), dtype='float32')
    dis_embedding0level = Embedding(output_dim=embedding_vector_length,
                                input_dim=max_features,
                                input_length=DIS_MAX_LEN,
                                weights=[embedding_matrix0],
                                trainable=args.trainable)(dis_in)
    dis_embedding1level = Embedding(output_dim=embedding_vector_length,
                                input_dim=max_features,
                                input_length=DIS_MAX_LEN,
                                weights=[embedding_matrix1],
                                trainable=args.trainable)(dis_in)
    dis_embedding2level = Embedding(output_dim=embedding_vector_length,
                                input_dim=max_features,
                                input_length=DIS_MAX_LEN,
                                weights=[embedding_matrix2],
                                trainable=args.trainable)(dis_in)
    dis_embedding3level = Embedding(output_dim=embedding_vector_length,
                                input_dim=max_features,
                                input_length=DIS_MAX_LEN,
                                weights=[embedding_matrix3],
                                trainable=args.trainable)(dis_in)
    #bn = BatchNormalization() #BN层就是为了让让每一层的值在一个有效范围内传递下去。1、加快网络的训练和收敛的速度 2、控制梯度爆炸防止梯度消失 3、防止过拟合
    #fusion_vector = concatenate([dis_embedding0level, dis_embedding1level, dis_embedding2level, dis_embedding3level])
    conv_result = []
    for i in range(3):
        channel_result = []
        #lstm = LSTM(units=1000, recurrent_activation='leaky_relu', dropout=0.1) 
        #lstm_out = lstm(fusion_vector)
        conv_layer = Conv1D(filter_number, filter_size, padding='same', activation='relu')
        conv0 = conv_layer(dis_embedding0level)
        conv1 = conv_layer(dis_embedding1level)
        conv2 = conv_layer(dis_embedding2level)
        conv3 = conv_layer(dis_embedding3level)
        pooling0 = GlobalMaxPooling1D()(conv0)
        pooling1 = GlobalMaxPooling1D()(conv1)
        pooling2 = GlobalMaxPooling1D()(conv2)
        pooling3 = GlobalMaxPooling1D()(conv3)
        channel_result.append(pooling0)
        channel_result.append(pooling1)
        channel_result.append(pooling2)
        channel_result.append(pooling3)
        allchannel = add(channel_result)
        conv_result.append(allchannel)
        #bn = BatchNormalization()
        #conv_result.append(lstm_out)

    demgras_in = Input(shape=(demgras_dim,), dtype='float32')
    print("demgras_dim:{}".format(demgras_dim)) #demgras_dim:11
    print("demgras_in.shape:{}".format(demgras_in.shape))#(None, 11) sigmoid
    dense_demgras = Dense(1000, activation='sigmoid')(demgras_in) #它将任意的值转换到 [0,1] 之间 BN层是将数据转换为均值为0，方差为1的正态分布
    conv_result.append(dense_demgras)#append the demographics information
    merge_out = concatenate(conv_result) #numpy.concatenate函数 主要作用:沿现有的某个轴对一系列数组进行拼接。
    merge_out = tf.expand_dims(merge_out, axis=0)
    x = se_block(merge_out)  # 接收SE返回值 #可行 但是平均误差突然间猛增 动荡 这个网络不错诶 虽然前期增加，但是后期整体趋势在下降，虽然偶尔动荡
    #x = eca_block(merge_out)  # 接收ECA输出结果 这个网络类似于上面这个网络 等会可以试试 和SENet相比大大减少了参数量，参数量等于一维卷积的kernel_size的大小
    print(x.shape)
    x=tf.squeeze(x,[0, 1])
    dense_out = Dense(1000, activation='relu', name='F1')(x)
    dpt = Dropout(dropout)(dense_out)
    dense_out = Dense(500, activation='relu', name='F2')(dpt) 
    dpt = Dropout(dropout)(dense_out)
    dense_out = Dense(100, activation='relu', name='F3')(dpt) #RReLU中的aji是一个在一个给定的范围内随机抽取的值，这个值在测试环节就会固定下来。
    dpt = Dropout(dropout)(dense_out)
    dense_out = Dense(50, activation='relu', name='F4')(dpt) #Leaky ReLU中的ai是固定的
    dpt = Dropout(dropout)(dense_out)
    #activation_function = keras.layers.advanced_activations.PReLU(init='zero', weights=None) 
    dense_out = Dense(2, activation='leaky_relu', name='F5')(dpt) #leaky_relu elu PReLU #PReLU中的ai是根据数据变化的#
    #dense_out = prelu(dense_out)
    dpt = Dropout(dropout)(dense_out)
    mode_out = Dense(1)(dpt)
    model = Model([dis_in,demgras_in], mode_out) 
    #rmsprop = optimizers.rmsprop(lr=lr, rho=0.9, epsilon=10-8, decay=0.0) 均方根传播(RMSProp) ;根据最近的权重梯度的平均值(例如变化的速度)来调整;这意味着该算法在线上和非平稳问题上表现良好(如:噪声)。
    Adam=tf.keras.optimizers.Adam(lr=0.0008, beta_1=0.9, beta_2=0.99, epsilon=1e-08, decay=0.0)
    #rmsprop tf.keras.losses.Huber()
    model.compile(loss='mean_squared_error', optimizer=Adam, metrics=['mae']) #tf.keras.losses.Huber()
    print(model.summary())
    return model

def train_ATTENTION_multi_channel_split_model(model, modelpath, X1_train,  demographics_train, y_train):
    #train_mat,valid_mat, train_y, valid_y, demographics_train, demographics_valid = train_test_split(X_train, y_train, demographics_train, test_size=0.05, random_state=seed)
    checkpointer = ModelCheckpoint(filepath="/content/gdrive/MyDrive/cnn_model/%s.hdf5"%(modelpath),verbose=1, save_best_only=True)
    earlystopper = EarlyStopping(monitor='val_loss', patience=30, verbose=1)#patience：能够容忍多少个epoch内都没有improvement。verbose：日志显示 verbose = 1 为输出进度条记录

    print('Training the ATTENTION_merge model....') 
    history =model.fit([X1_train,  demographics_train], y_train, epochs=200, batch_size=args.batchsize, shuffle=True,
              validation_split=0.1,
              callbacks=[checkpointer,earlystopper], # 尝试关闭早停
              verbose=1)
    print("4、开始画图：损失")
    from matplotlib import rcParams
    font2 = {'family': 'Times New Roman',
             'weight': 'normal',
               'size': 15,}
    plt.figure()
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss',fontsize=15)
    plt.xlabel('Epoch',font2,verticalalignment='top') # fontsize=14, 
    plt.ylabel('Loss',font2, horizontalalignment='center') #fontsize=14, 
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.legend(loc=1,fontsize=10,frameon=True)
    plt.savefig('/content/gdrive/MyDrive/LOSS.png',dpi=500,bbox_inches = 'tight')
 
def test_ATTENTION_multi_channel_split_model(model, modelpath, X1_test,  demographics_test, y_test,index):
    print('Testing model...') 
    model.load_weights("/content/gdrive/MyDrive/cnn_model/%s.hdf5"%(modelpath))
    y_pred = model.predict([X1_test, demographics_test], batch_size=args.batchsize, verbose=1)
    print("y_pred:{}".format(y_pred.shape))
    r2,rmse = evaluation(y_test, y_pred)
    print("r2:",r2,"rmse:",rmse)
    #画图1
    name = 'ATTENTION_MG_split%d'%(index)
    costplots(y_test, y_pred, r2, rmse, name)
    #画图2
    if args.isdays:
        daysplots(y_test, y_pred, r2, rmse, name)
    else:
        costplots(y_test, y_pred, r2, rmse, name)
    #画图3
    print("!!开始画图：预测值与真实值")
    print("y_test:",y_test.shape)
    print("y_pred:",y_pred.shape)
    print("y_pred.ndim:",y_pred.ndim)
    print("y_test.ndim:",y_test.ndim)
    from matplotlib import rcParams
    font2 = {'family': 'Times New Roman',
             'weight': 'normal',
               'size': 15,}

    plt.figure()
    plt.plot(list(range(1569)),y_test,label='True Value')
    plt.plot(list(range(1569)),y_pred,label='Predicted Value')
    b=np.squeeze(y_test)
    c=np.squeeze(y_pred)
    plt.fill_between(list(range(1569)),b,c,color='g',alpha=.25)
    plt.title('Predicted and true values of medical costs',fontsize=13)
    plt.xlabel('Series',fontsize=13, verticalalignment='top') #
    plt.ylabel('Medical cost',fontsize=13,horizontalalignment='center') #
    plt.xticks(fontsize=13)
    plt.yticks(fontsize=13)
    plt.legend(loc=1,fontsize=10,frameon=True)
    plt.savefig('/content/gdrive/MyDrive/Attention_曲线图.png',dpi=500,bbox_inches = 'tight')
    #画散点图
    plt.figure()#figsize=(15,5)
    plt.plot(y_test,'rs',label='True value')
    plt.plot(y_pred,'go',label='Predict value')
    plt.title('Predicted and true values of medical costs',fontsize=13)
    plt.xticks(fontsize=13)
    plt.yticks(fontsize=13)
    plt.xlabel('Series',fontsize=13,)
    plt.ylabel('Medical Cost',fontsize=13)
    plt.legend(loc=1,fontsize=10,frameon=True)
    plt.savefig('/content/gdrive/MyDrive/Attention_散点图.png',dpi=500,bbox_inches = 'tight')
    #画：测试集的预测损失
    plt.figure()
    a=y_test-y_pred
    plt.plot(a, label='Loss')
    plt.title('Predicted loss of the proposed model',fontsize=13)
    plt.xlabel('Series',fontsize=10, verticalalignment='top') #
    plt.ylabel('Loss value',fontsize=10,horizontalalignment='center') #
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.savefig('/content/gdrive/MyDrive/Predicted_LOSS.png',dpi=500,bbox_inches = 'tight')

### **1.1.4 开始预测**

In [5]:
!pip install tsne

     |████████████████████████████████| 547 kB 8.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for tsne: filename=tsne-0.3.1-cp37-cp37m-linux_x86_64.whl size=260507 sha256=57d860cb9db7e6eaf880cc8661fe0b67adefb6b7bb9f3c637eedc8e45b233c56
  Stored in directory: /root/.cache/pip/wheels/b9/a7/9d/d09b0eef66f28be09470e0e18629ae08aed772497b218f84f3
Successfully built tsne


数据准备

In [7]:
path = 'weighted_%s:%s_%sdays_%sinit_%strainable_%fdpt_%flr_%dfz_%dfn_%dmaxlen_%ddim_%sfilter_%stransform %sbatch_%swindow'%(MID, SID, args.isdays, args.init, args.trainable, args.dpt, args.lr, args.fz, args.fn, args.maxlen, args.dim, args.filter, args.transform,args.batchsize,args.window)

if os.path.isfile('/content/gdrive/MyDrive/dataclean.df'):
    data = pkl.load(open('/content/gdrive/MyDrive/dataclean.df','rb'))
else:
    data = DataClean()
rawseqs, cost, days, demographics, disease, main_dis = ToRawList(data) #rawseqs就是seqs
#print("rawseqs:",rawseqs)

seqs1levels = []
seqs2levels = []
seqs3levels = []
for seqs in rawseqs:
    levels1 = []
    levels2 = []
    levels3 = []
    for code in seqs:
        levels1.append(code[0:3])
        if len(re.findall(pattern, code))== 1:
            levels2.append(code[0:5])
            levels3.append(code[0:6])
        else:
            levels2.append(code[0:4])
            levels3.append(code[0:5])
    seqs1levels.append(levels1)
    seqs2levels.append(levels2)
    seqs3levels.append(levels3)

print('开始将疾病诊断代码转换为索引')
code_index = token_to_index(rawseqs)# transform code into index 将代码转换为索引
index_code = dict([(kv[1], kv[0]) for kv in code_index.items()])# validate the code_index 验证代码_索引
#print [[index_code[index] for index in item ] for item in seqs[0:2]]

idseqs = [[code_index[code] for code in item] for item in rawseqs]
iddis = [[code_index[code] for code in item] for item in disease]
idmain_dis = [[code_index[code] for code in item] for item in main_dis]
#print(main_dis)
#"""
print('开始形成词嵌入向量')
print(disease)
print(disease[1:]) #list 15684
word2vec_model(disease,0)
index_embedding = get_index_embedding(code_index, 0)
embedding_matrix0 = get_trained_embedding(index_embedding)
word2vec_model(disease, 1)
index_embedding1 = get_index_embedding(code_index, 1)
embedding_matrix1 = get_trained_embedding(index_embedding1)
word2vec_model(disease, 2)
index_embedding2 = get_index_embedding(code_index, 2)
embedding_matrix2 = get_trained_embedding(index_embedding2)
word2vec_model(disease, 3)
index_embedding3 = get_index_embedding(code_index, 3)
embedding_matrix3 = get_trained_embedding(index_embedding3)

main_code = []
for item in idmain_dis:
    for ind in item:
        main_code.append(embedding_matrix0[1][ind])

maincodemat = np.array(main_code)

nb_words = len(index_code) # code_index starting from 1
max_features = nb_words + 1 
n_seqs = len(idseqs)

MAX_LEN = args.maxlen      # the max len is 21 #这里没有治疗编码，所以最大长度为以记录为单位
seqs = pad_sequences(idseqs, maxlen=MAX_LEN)
DIS_MAX_LEN = 17  #其实这里需要改成以记录为单位才对 DIS_MAX_LEN = 11 SUR_MAX_LEN = 10 maxlen:21
disease = pad_sequences(iddis, maxlen=DIS_MAX_LEN)


X = np.array(seqs)
X1 = np.array(disease)
print("X(seqs):",X.shape,"X1(disease):",X1.shape)
print("seqs:",seqs.shape)


"""
if args.test:
    modelpath = 'MG_merge' + path
    demgras_dim = 3
    model= multi_channel_split_model(demgras_dim)
    extract_patientvec(model, modelpath, disease, surgery, demographics)
"""
#demographics = np.hstack((demographics, maincodemat))
if args.isdays:
    y = np.asarray(days, dtype='float32')
else:
    y = np.asarray(cost,dtype='float32')

if args.transform:
    print("np.log")
    y = np.log(y)
print(X, demographics)

comm_inds= filter_test(X, index_code, 4)
comm_X = X[comm_inds]
comm_X1 = X1[comm_inds]
comm_demographics = demographics[comm_inds]
comm_y = y[comm_inds]
print("comm_X:",comm_X,"comm_X1:",comm_X1,"comm_y:",comm_y)

print('Spliting train, test parts...')
X_train, X_test, X1_train, X1_test, y_train, y_test, demographics_train, demographics_test = train_test_split(comm_X, comm_X1,comm_y, comm_demographics, test_size=0.1, random_state=90)
print('X_train:{}'.format(X_train.shape))
print('X_test:{}'.format(X_test.shape))
print('X1_train:{}'.format(X1_train.shape))
print('X1_test:{}'.format(X1_test.shape))
print('y_train:{}'.format(y_train.shape))
print('y_test:{}'.format(y_test.shape))
print('demographics_train:{}'.format(demographics_train.shape))
print('demographics_test:{}'.format(demographics_test.shape))

#y_train=np.log(y_train)
#y_test=np.log(y_test)
print("y_test:",y_test)
  
demgras_dim = demographics.shape[1] #图像的水平尺寸 
print("demgras_dim:{}".format(demgras_dim)) #demgras_dim:10

开始步骤2:形成原始单词清单
开始将疾病诊断代码转换为索引
开始步骤3:给单词编号,形成单词-编号的字典
开始形成词嵌入向量
[['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['O24.901'], ['I67.801'], ['E71.001'], ['H81.904'], ['E10.901'], ['E10.901'], ['I10.X02'], ['I25.101'], ['I25.101'], ['I63.902'], ['E10.901'], ['E10.901'], ['E10.901'], ['H04.401'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['B21.253'], ['E10.901'], ['E10.901'], ['I25.101'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['I10.X01'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['E10.901'], ['I10.X01'], ['I10.X01'], ['E10.901'], ['E10.901'], 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:327: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:986: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  FutureWarning,


[t-SNE] KL divergence after 250 iterations with early exaggeration: 76.328392
[t-SNE] KL divergence after 1000 iterations: 1.637868
开始第4步
开始get_trained_embedding
开始63/3:开始word2vec


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:207: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:208: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


model: Word2Vec(vocab=242, size=900, alpha=0.025)
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 242 samples in 0.000s...
[t-SNE] Computed neighbors for 242 samples in 0.010s...
[t-SNE] Computed conditional probabilities for sample 242 / 242
[t-SNE] Mean sigma: 0.001710


...

model: Word2Vec(vocab=242, size=900, alpha=0.025)
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 242 samples in 0.001s...
[t-SNE] Computed neighbors for 242 samples in 0.010s...
[t-SNE] Computed conditional probabilities for sample 242 / 242
[t-SNE] Mean sigma: 0.001710


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:327: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:986: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  FutureWarning,


[t-SNE] KL divergence after 250 iterations with early exaggeration: 73.533821
[t-SNE] KL divergence after 1000 iterations: 1.639425
开始第4步
开始get_trained_embedding
X(seqs): (15684, 17) X1(disease): (15684, 17)
seqs: (15684, 17)
np.log
[[  0   0   0 ...   7   1   8]
 [  0   0   0 ...  11   1   8]
 [  0   0   0 ...  11   1   8]
 ...
 [  0   0   0 ...  11 112   8]
 [  0   0   0 ...  11 112   8]
 [  0   0   0 ...   3 111  49]] [[ 0. 63.  1. ...  5.  2.  0.]
 [ 0. 81.  1. ... 11.  7.  0.]
 [ 0. 43.  1. ... 11.  7.  0.]
 ...
 [ 0. 68.  0. ... 12.  7. 34.]
 [ 0. 62.  0. ... 12.  7. 34.]
 [ 0. 59.  0. ... 12.  1. 33.]]
开始步骤8:选择罕见集样本用于测试集...，这一步其实是将所有数据15684都用于了测试集


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:207: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:208: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


comm_inds:15684
comm_X: [[  0   0   0 ...   7   1   8]
 [  0   0   0 ...  11   1   8]
 [  0   0   0 ...  11   1   8]
 ...
 [  0   0   0 ...  11 112   8]
 [  0   0   0 ...  11 112   8]
 [  0   0   0 ...   3 111  49]] comm_X1: [[ 0  0  0 ...  0  0  8]
 [ 0  0  0 ...  0  0  8]
 [ 0  0  0 ...  0  0  8]
 ...
 [ 0  0  0 ...  0  0  8]
 [ 0  0  0 ...  0  0  8]
 [ 0  0  0 ...  0  0 49]] comm_y: [[8.716206]
 [8.190104]
 [8.262434]
 ...
 [5.70374 ]
 [5.70374 ]
 [5.858118]]
Spliting train, test parts...
X_train:(14115, 17)
X_test:(1569, 17)
X1_train:(14115, 17)
X1_test:(1569, 17)
y_train:(14115, 1)
y_test:(1569, 1)
demographics_train:(14115, 10)
demographics_test:(1569, 10)
y_test: [[7.9070454]
 [8.753389 ]
 [5.70374  ]
 ...
 [8.566658 ]
 [5.70374  ]
 [5.70374  ]]
demgras_dim:10


### **有时间间隔**

In [16]:
#=============================== 第四部分 开始预测 =====================================================
r2_CNN_val=[]
r2_CNN_test=[]
for i in range(10):
  print('第%s次训练'%i)
  
  modelpath = '有时间间隔_5月16日(下午)_'+'第%s次训练_'%i+'MG_merge' + path # %i
  #定义模型
  model= ATTENTION_multi_channel_split_model(demgras_dim) 
  #划分数据
  X1_train_1,X1_val_1, demographics_train_1,demographics_val_1,y_train_1,y_val_1=train_test_split( X1_train, demographics_train, y_train, test_size=0.1, random_state=90) 
  if not args.test:
      train_ATTENTION_multi_channel_split_model(model, modelpath, X1_train_1, demographics_train_1, y_train_1)
  extract_patientvec(model, modelpath, disease, demographics)
  #拟合、预测
  y_CNN_hat = model.predict([X1_val_1,demographics_val_1])

  r2,rmse = evaluation(y_val_1, y_CNN_hat) #验证集的拟合优度
  R21=r2_score(y_val_1, y_CNN_hat)
  print(i,R21)
  r2_CNN_val.append(R21)
  
  test_ATTENTION_multi_channel_split_model(model, modelpath, X1_test, demographics_test,y_test,3)
  model.load_weights("/content/gdrive/MyDrive/cnn_model/%s.hdf5"%(modelpath))
  y_CNN_pred = model.predict([X1_test, demographics_test], batch_size=args.batchsize, verbose=1)
  r22,rmse = evaluation(y_test, y_CNN_pred)
  print("r2:",r22,"rmse:",rmse)
  r2_CNN_test.append(r22)


第0次训练
demgras_dim:10
demgras_in.shape:(None, 10)
通道数:1300
sigmoid激活，权重归一化: KerasTensor(type_spec=TensorSpec(shape=(1, 1, 1, 1300), dtype=tf.float32, name=None), name='tf.math.sigmoid_13/Sigmoid:0', description="created by layer 'tf.math.sigmoid_13'")
(1, 1, None, 1300)
Model: "model_73"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_27 (InputLayer)          [(None, 17)]         0           []                               
                                                                                                  
 embedding_52 (Embedding)       (None, 17, 900)      649800      ['input_27[0][0]']               
                                                                                                  
 embedding_53 (Embedding)       (None, 17, 900)      649800      ['input_27[0][0]']               
                   

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


                                                                                                  
 concatenate_13 (Concatenate)   (None, 1300)         0           ['add_39[0][0]',                 
                                                                  'add_40[0][0]',                 
                                                                  'add_41[0][0]',                 
                                                                  'dense_52[0][0]']               
                                                                                                  
 tf.expand_dims_13 (TFOpLambda)  (1, None, 1300)     0           ['concatenate_13[0][0]']         
                                                                                                  
 global_average_pooling1d_13 (G  (1, 1300)           0           ['tf.expand_dims_13[0][0]']      
 lobalAveragePooling1D)                                                                           
          

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:144: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


491/491 [==============================] - 3s 5ms/step
(15684, 100)
491/491 [==============================] - 3s 5ms/step
(15684, 50)
491/491 [==============================] - 3s 5ms/step
(15684, 2)
y_test:(1412, 1)
y_pred:(1412, 1)
float32
MSE:0.07584987580776215, RMSE:0.2754085659980774, MAE:0.11753012984991074, R2:0.9170526773382071
0 0.9170526773382071
Testing model...
13/13 [==============================] - 0s 9ms/step
y_pred:(1569, 1)
y_test:(1569, 1)
y_pred:(1569, 1)
float32
MSE:0.08113591372966766, RMSE:0.28484365344047546, MAE:0.11575396358966827, R2:0.9096758243606976
r2: 0.9096758243606976 rmse: 0.28484365
开始步骤92/2:预测可视化-医疗费用


...

开始步骤92/2:预测可视化-医疗费用


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:457: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:468: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:181: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warnin

!!开始画图：预测值与真实值
y_test: (1569, 1)
y_pred: (1569, 1)
y_pred.ndim: 2
y_test.ndim: 2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:195: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:206: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


13/13 [==============================] - 0s 16ms/step
y_test:(1569, 1)
y_pred:(1569, 1)
float32
MSE:0.07523752748966217, RMSE:0.27429458498954773, MAE:0.11420423537492752, R2:0.9162421799026989
r2: 0.9162421799026989 rmse: 0.27429458


In [17]:
r2_CNN_val

[0.9170526773382071,
 0.910960730842986,
 0.922691272816625,
 0.9197774991282821,
 0.9159260300487481,
 0.9190478082128568,
 0.9200917263247873,
 0.921309073588352,
 0.9205265706080363,
 0.9209671109789732]

In [18]:
r2_CNN_test

[0.9096758243606976,
 0.9069428181583941,
 0.9197171100459803,
 0.916395433172939,
 0.9136934311893576,
 0.9185873098366197,
 0.918008913017388,
 0.9212146105790974,
 0.9173297577883059,
 0.9162421799026989]

### **没有时间间隔**

In [12]:
#=============================== 第四部分 没有时间间隔——开始预测 =====================================================
r2_CNN_val=[]
r2_CNN_test=[]
for i in range(10):
  print('第%s次训练'%i)
  
  modelpath = '没有时间间隔_5月16日(下午)_'+'第%s次训练_'%i+'MG_merge' + path # %i
  #定义模型
  model= ATTENTION_multi_channel_split_model(demgras_dim) 
  #划分数据
  X1_train_1,X1_val_1, demographics_train_1,demographics_val_1,y_train_1,y_val_1=train_test_split( X1_train, demographics_train, y_train, test_size=0.1, random_state=90) 
  if not args.test:
      train_ATTENTION_multi_channel_split_model(model, modelpath, X1_train_1, demographics_train_1, y_train_1)
  extract_patientvec(model, modelpath, disease, demographics)
  #拟合、预测
  y_CNN_hat = model.predict([X1_val_1,demographics_val_1])

  r2,rmse = evaluation(y_val_1, y_CNN_hat) #验证集的拟合优度
  R21=r2_score(y_val_1, y_CNN_hat)
  print(i,R21)
  r2_CNN_val.append(R21)
  
  test_ATTENTION_multi_channel_split_model(model, modelpath, X1_test, demographics_test,y_test,3)
  model.load_weights("/content/gdrive/MyDrive/cnn_model/%s.hdf5"%(modelpath))
  y_CNN_pred = model.predict([X1_test, demographics_test], batch_size=args.batchsize, verbose=1)
  r22,rmse = evaluation(y_test, y_CNN_pred)
  print("r2:",r22,"rmse:",rmse)
  r2_CNN_test.append(r22)

第0次训练
demgras_dim:10
demgras_in.shape:(None, 10)
通道数:1300
sigmoid激活，权重归一化: KerasTensor(type_spec=TensorSpec(shape=(1, 1, 1, 1300), dtype=tf.float32, name=None), name='tf.math.sigmoid_3/Sigmoid:0', description="created by layer 'tf.math.sigmoid_3'")
(1, 1, None, 1300)
Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 17)]         0           []                               
                                                                                                  
 embedding_12 (Embedding)       (None, 17, 900)      649800      ['input_7[0][0]']                
                                                                                                  
 embedding_13 (Embedding)       (None, 17, 900)      649800      ['input_7[0][0]']                
                     

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


                                                                                                  
 tf.math.sigmoid_3 (TFOpLambda)  (1, 1, 1, 1300)     0           ['dense_14[0][0]']               
                                                                                                  
 multiply_3 (Multiply)          (1, 1, None, 1300)   0           ['tf.expand_dims_3[0][0]',       
                                                                  'tf.math.sigmoid_3[0][0]']      
                                                                                                  
 tf.compat.v1.squeeze_3 (TFOpLa  (None, 1300)        0           ['multiply_3[0][0]']             
 mbda)                                                                                            
                                                                                                  
 F1 (Dense)                     (None, 1000)         1301000     ['tf.compat.v1.squeeze_3[0][0]'] 
          

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:144: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


491/491 [==============================] - 3s 5ms/step
(15684, 100)
491/491 [==============================] - 3s 5ms/step
(15684, 50)
491/491 [==============================] - 3s 5ms/step
(15684, 2)
y_test:(1412, 1)
y_pred:(1412, 1)
float32
MSE:0.07909499108791351, RMSE:0.281238317489624, MAE:0.11872976273298264, R2:0.9135039079983848
0 0.9135039079983848
Testing model...
13/13 [==============================] - 0s 10ms/step
y_pred:(1569, 1)
y_test:(1569, 1)
y_pred:(1569, 1)
float32
MSE:0.08110012114048004, RMSE:0.2847808301448822, MAE:0.1184941902756691, R2:0.9097156799067575
r2: 0.9097156799067575 rmse: 0.28478083
开始步骤92/2:预测可视化-医疗费用


...

开始步骤92/2:预测可视化-医疗费用


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:457: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:468: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:181: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warnin

!!开始画图：预测值与真实值
y_test: (1569, 1)
y_pred: (1569, 1)
y_pred.ndim: 2
y_test.ndim: 2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:195: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:206: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


13/13 [==============================] - 0s 16ms/step
y_test:(1569, 1)
y_pred:(1569, 1)
float32
MSE:0.07312426716089249, RMSE:0.27041497826576233, MAE:0.11622720956802368, R2:0.918594758214431
r2: 0.918594758214431 rmse: 0.27041498


In [13]:
r2_CNN_val

[0.9135039079983848,
 0.9171583225677296,
 0.9232899830262921,
 0.9099373038593238,
 0.9195682834084178,
 0.9174504361181719,
 0.9189608483056998,
 0.922871844232307,
 0.919464255721107,
 0.9211026264086222]

In [14]:
r2_CNN_test

[0.9097156799067575,
 0.9133483565999173,
 0.9199765740763938,
 0.9078222057681373,
 0.9151527028515862,
 0.9139237875328569,
 0.9174715523063106,
 0.91360068106163,
 0.9175436190007902,
 0.918594758214431]

### **1.1.5 Grad-CAM可视化**

#### **第一次尝试**

In [ ]:
model.summary()

Model: "model_36"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 3)]          0           []                               
                                                                                                  
 embedding_24 (Embedding)       (None, 3, 900)       649800      ['input_13[0][0]']               
                                                                                                  
 embedding_25 (Embedding)       (None, 3, 900)       649800      ['input_13[0][0]']               
                                                                                                  
 embedding_26 (Embedding)       (None, 3, 900)       649800      ['input_13[0][0]']               
                                                                                           

#### **第二次尝试**

In [ ]:
import keras
# import cv2
import numpy as np
import keras.backend as K
 
#from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import load_img,img_to_array
#K.set_learning_phase(1) #set learning phase
# 注释掉的是针对输入为图片的情况 
# 需根据自己情况修改1.训练好的模型路径
weight_file_dir = '/content/gdrive/MyDrive/cnn_model/5月13日(1)_第0次训练_MG_mergeweighted_55:38_Falsedays_Trueinit_Truetrainable_0.000000dpt_0.005000lr_3fz_100fn_17maxlen_900dim_Truefilter_Truetransform 128batch_10window.hdf5'


# x为你的输入，现在的代码逻辑是针对一条数据的
# 如果一次计算很多条就需要考虑多出来的那个维度了
x = [X1_test, demographics_test]
print("X1_test.ndim:",X1_test.ndim)
print("demographics_test.ndim:",demographics_test.ndim)
print("X1_test.shape:",X1_test.shape)
x1=np.squeeze(X1_test)
x2=np.squeeze(demographics_test)
print("x1.ndim:",x1.ndim)
print("x2.ndim:",x2.ndim)
print("x1.shape:",x1.shape)
print("x2.shape:",x2.shape)
x3=[x1,x2]
print(len(x3))

modelpath = '5月13日(1)_'+'第0次训练_'+'MG_merge' + path

#test_ATTENTION_multi_channel_split_model(model, modelpath, X1_test, demographics_test,y_test,3) #预测模型
model.load_weights("/content/gdrive/MyDrive/cnn_model/%s.hdf5"%(modelpath))

y_CNN_pred = model.predict([x1,x2], batch_size=1, verbose=1) #开始预测
y_CNN_pred1=tf.convert_to_tensor(y_CNN_pred) #将预测结果转换为张量
class_output = y_CNN_pred1

last_conv_layer = model.get_layer("conv1d_2") # 需根据自己情况修改2. 把block5_conv3改成需要关注的网络层名字


X1_test.ndim: 2
demographics_test.ndim: 2
X1_test.shape: (1569, 3)
x1.ndim: 2
x2.ndim: 2
x1.shape: (1569, 3)
x2.shape: (1569, 10)
2
1569/1569 [==============================] - 10s 6ms/step


In [ ]:
model.output

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dense_27')>

In [ ]:
class_output

<tf.Tensor: shape=(1569, 1), dtype=float32, numpy=
array([[5.6822214],
       [5.6830072],
       [5.717518 ],
       ...,
       [5.6804886],
       [5.6824117],
       [5.6923018]], dtype=float32)>

In [ ]:
last_conv_layer.output

<KerasTensor: shape=(None, 3, 300) dtype=float32 (created by layer 'conv1d_19')>

In [ ]:
model.output

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dense_27')>

In [ ]:
model.input

[<KerasTensor: shape=(None, 3) dtype=float32 (created by layer 'input_13')>,
 <KerasTensor: shape=(None, 10) dtype=float32 (created by layer 'input_14')>]

In [ ]:
# Grad-CAM 算法实现
import tensorflow.keras.backend as K
with tf.GradientTape() as gtape: #keras.backend.gradients
    print("class_output:",class_output)
    print(last_conv_layer.output)
    #print(model.output[0])
    #aa=tf.squeeze(last_conv_layer.output,axis=0)
    #print(aa[0])
    grads = gtape.gradient(model.output,last_conv_layer.output)[0] # last_conv_layer.output
pooled_grads = K.mean(grads,axis=(0))
iterate = K.function([model.input],[pooled_grads,last_conv_layer.output[0]])
pooled_grads_value, conv_layer_output_value = iterate([x])
# 需根据自己情况修改3. 100是此层卷积层的通道数，根据自己情况修改
# 因为你的卷积是一维卷积，所以和二维的图像卷积相比，输出也自然少了一维
for i in range(100):
    # conv_layer_output_value[:,:,i] *= pooled_grads_value[i]
    conv_layer_output_value[:,i] *= pooled_grads_value[i]
 
heatmap = np.mean(conv_layer_output_value, axis=-1)
heatmap = np.maximum(heatmap,0)
heatmap /= np.max(heatmap)

# 下面这里相当于把热力图加到原始输入图像中
# 因为你的输入不是图片，所以可考虑直接打印热力图矩阵？
# 和输入同形状，意义为输入各个部分的重要程度
heatmap = np.resize(heatmap, x.shape[0])
print(heatmap)

class_output: tf.Tensor(
[[5.7471347]
 [5.737805 ]
 [5.7776055]
 ...
 [5.7335706]
 [5.7313743]
 [5.750322 ]], shape=(1569, 1), dtype=float32)
KerasTensor(type_spec=TensorSpec(shape=(None, 3, 300), dtype=tf.float32, name=None), name='conv1d_2/Relu:0', description="created by layer 'conv1d_2'")


AttributeError: ignored

In [ ]:
class_output.dtype

tf.float32

In [ ]:
last_conv_layer.output

<KerasTensor: shape=(None, 3, 300) dtype=float32 (created by layer 'conv1d_5')>

#### **第三次尝试**

In [ ]:
import keras
import numpy as np
import keras.backend as K
 
#from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import load_img,img_to_array
K.set_learning_phase(1) #set learning phase

# 注释掉的是针对输入为图片的情况 
# 需根据自己情况修改1.训练好的模型路径
weight_file_dir = '/content/gdrive/MyDrive/cnn_model/5月13日(1)_第0次训练_MG_mergeweighted_35:39_Falsedays_Trueinit_Truetrainable_0.010000dpt_0.005000lr_3fz_300fn_17maxlen_900dim_Truefilter_Truetransform 128batch_10window.hdf5'
model = keras.models.load_model(weight_file_dir)
x = [X1_test, demographics_test]
x = [np.array([x[0][0]]),np.array([x[1][0]])] # x为你的输入，现在的代码逻辑是针对一条数据的 # 如果一次计算很多条就需要考虑多出来的那个维度了
class_output = model.predict(x)

/usr/local/lib/python3.7/dist-packages/keras/backend.py:450: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


ValueError: ignored

In [ ]:
class_output

array([[5.7459364],
       [5.738635 ],
       [5.723418 ],
       ...,
       [5.738988 ],
       [5.7416162],
       [5.742443 ]], dtype=float32)

In [ ]:
# 需根据自己情况修改2. 把block5_conv3改成需要关注的网络层名字
# last_conv_layer = model.get_layer("block5_conv3")
last_conv_layer = model.get_layer("conv1d_5")
with tf.GradientTape() as gtape: #keras.backend.gradients
    print("class_output:",class_output)
    print(last_conv_layer.output)
    grads = gtape.gradient(model.output,last_conv_layer.output)[0]
#grads = K.gradients(class_output,last_conv_layer.output)[0]
pooled_grads = K.mean(grads,axis=(0))
iterate = K.function([model.input],[pooled_grads,last_conv_layer.output[0]])
pooled_grads_value, conv_layer_output_value = iterate([x])
# 需根据自己情况修改3. 100是此层卷积层的通道数，根据自己情况修改
# 因为你的卷积是一维卷积，所以和二维的图像卷积相比，输出也自然少了一维
for i in range(100):
    # conv_layer_output_value[:,:,i] *= pooled_grads_value[i]
    conv_layer_output_value[:,i] *= pooled_grads_value[i]
 
heatmap = np.mean(conv_layer_output_value, axis=-1)
heatmap = np.maximum(heatmap,0)
heatmap /= np.max(heatmap)
 
# 下面这里相当于把热力图加到原始输入图像中
# 因为你的输入不是图片，所以可考虑直接打印热力图矩阵？
# 和输入同形状，意义为输入各个部分的重要程度
heatmap = np.resize(heatmap, x.shape[0])
print(heatmap)


class_output: [[5.7459364]
 [5.738635 ]
 [5.723418 ]
 ...
 [5.738988 ]
 [5.7416162]
 [5.742443 ]]
KerasTensor(type_spec=TensorSpec(shape=(None, 3, 300), dtype=tf.float32, name=None), name='conv1d_5/Relu:0', description="created by layer 'conv1d_5'")


AttributeError: ignored

## **1.2 原卷积神经网络模型预测**

### **1.2.1 原卷积神经网络模型预测(1)**

In [ ]:
r2_SG_merge=[]
seed = 1234
def test_single_channel_merge_model(model, modelpath,X_test,demographics_test,y_test, index):
    print('Testing model...') 
    model.load_weights("./cnn_model/%s.hdf5"%(modelpath))
    y_pred = model.predict([X_test, demographics_test], batch_size=args.batchsize, verbose=1)
    r2,rmse = evaluation(y_test, y_pred)
    print(r2)
    r2_SG_merge.append(r2)
    name = 'SG_merge%d'%(index)
    if args.isdays:
        daysplots(y_test, y_pred, r2, rmse, name)
    else:
        costplots(y_test, y_pred, r2, rmse, name)

r2_SG_split=[]
def test_single_channel_split_model(model, modelpath, X1_test,  demographics_test, y_test,index):
    print('Testing model...') 
    model.load_weights("./cnn_model/%s.hdf5"%(modelpath))
    y_pred = model.predict([X1_test,  demographics_test], batch_size=args.batchsize, verbose=1)
    r2,rmse = evaluation(y_test, y_pred)
    r2_SG_split.append(r2)
    name = 'SG_merge%d'%(index)
    if args.isdays:
        daysplots(y_test, y_pred, r2, rmse, name)
    else:
        costplots(y_test, y_pred, r2, rmse, name)

r2_MG_merge=[]
def test_multi_channel_split_model(model, modelpath, X1_test,  demographics_test, y_test,index):
    print('Testing model...') 
    model.load_weights("./cnn_model/%s.hdf5"%(modelpath))
    y_pred = model.predict([X1_test, demographics_test], batch_size=args.batchsize, verbose=1)
    print("y_pred:{}".format(y_pred.shape))
    r2,rmse = evaluation(y_test, y_pred)
    r2_MG_merge.append(r2)
    name = 'MG_split%d'%(index)
    if args.isdays:
        daysplots(y_test, y_pred, r2, rmse, name)
    else:
        costplots(y_test, y_pred, r2, rmse, name)



for i in range(10):
  modelpath = 'SG_merge' + path
  model = single_channel_merge_model(demgras_dim)
  if not args.test:
      train_single_channel_merge_model(model, modelpath, X_train, demographics_train, y_train)
  print(f,'testing on the testing datasets.....')
  test_single_channel_merge_model(model, modelpath, X_test, demographics_test, y_test, 3)
  print(f,'testing on the filtered testing datasets.....')
  print(r2_SG_merge)


for i in range(10):
  modelpath = 'SG_split' + path
  model = single_channel_split_model(demgras_dim)
  if not args.test:
      train_single_channel_split_model(model, modelpath, X1_train, demographics_train, y_train)
  print(f,'testing on the testing datasets.....')
  test_single_channel_split_model(model, modelpath, X1_test,  demographics_test,y_test,3)
  #print(f,'testing on the filtered testing datasets.....')
  print(r2_SG_split)


for i in range(10):
  modelpath = 'MG_merge' + path
  model= multi_channel_split_model(demgras_dim)
  if not args.test:
      train_multi_channel_split_model(model, modelpath, X1_train, demographics_train, y_train)
  extract_patientvec(model, modelpath, disease, demographics)
  print(f,'testing on the testing datasets.....')
  test_multi_channel_split_model(model, modelpath, X1_test, demographics_test,y_test,3)
  print(r2_MG_merge)

开始模型1:单尺度融合模型
initialize embedding layer with pre-training vectors
依据预训练向量 初始化嵌入层
embedding layers trainalble True
添加人口统计信息，融合
Model: "model_120"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_41 (InputLayer)          [(None, 17)]         0           []                               
                                                                                                  
 embedding_80 (Embedding)       (None, 17, 900)      649800      ['input_41[0][0]']               
                                                                                                  
 conv1d_60 (Conv1D)             (None, 17, 100)      270100      ['embedding_80[0][0]']           
                                                                                                  
 conv1d_61 (Conv1D)             (None, 17, 100)      270100   

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


None
Training the merge model....
Epoch 1/100
25/25 [==============================] - ETA: 0s - loss: 3.1465 - mae: 1.0259
Epoch 1: val_loss improved from inf to 0.76035, saving model to ./cnn_model/SG_merge/tmp/word2vec.model.hdf5
25/25 [==============================] - 3s 54ms/step - loss: 3.1465 - mae: 1.0259 - val_loss: 0.7604 - val_mae: 0.5859
Epoch 2/100
23/25 [==========================>...] - ETA: 0s - loss: 0.6987 - mae: 0.5482
Epoch 2: val_loss improved from 0.76035 to 0.61641, saving model to ./cnn_model/SG_merge/tmp/word2vec.model.hdf5
25/25 [==============================] - 1s 31ms/step - loss: 0.6958 - mae: 0.5479 - val_loss: 0.6164 - val_mae: 0.5126
Epoch 3/100
23/25 [==========================>...] - ETA: 0s - loss: 0.5893 - mae: 0.5105
Epoch 3: val_loss improved from 0.61641 to 0.52411, saving model to ./cnn_model/SG_merge/tmp/word2vec.model.hdf5
25/25 [==============================] - 1s 29ms/step - loss: 0.5857 - mae: 0.5080 - val_loss: 0.5241 - val_mae: 0.4834
E

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:689: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


/content/gdrive/MyDrive/15684_new_2.csv testing on the filtered testing datasets.....
[0.8404397759662624]
开始模型1:单尺度融合模型
initialize embedding layer with pre-training vectors
依据预训练向量 初始化嵌入层
embedding layers trainalble True
添加人口统计信息，融合
Model: "model_121"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_43 (InputLayer)          [(None, 17)]         0           []                               
                                                                                                  
 embedding_81 (Embedding)       (None, 17, 900)      649800      ['input_43[0][0]']               
                                                                                                  
 conv1d_63 (Conv1D)             (None, 17, 100)      270100      ['embedding_81[0][0]']           
                                                      

...

demgras_dim:10
demgras_in.shape:(None, 10)
Model: "model_274"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_159 (InputLayer)         [(None, 17)]         0           []                               
                                                                                                  
 embedding_196 (Embedding)      (None, 17, 900)      649800      ['input_159[0][0]']              
                                                                                                  
 embedding_197 (Embedding)      (None, 17, 900)      649800      ['input_159[0][0]']              
                                                                                                  
 embedding_198 (Embedding)      (None, 17, 900)      649800      ['input_159[0][0]']              
                                               

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


None
Training the merge model....
Epoch 1/100
25/25 [==============================] - ETA: 0s - loss: 5.1181 - mae: 1.5556
Epoch 1: val_loss improved from inf to 0.78736, saving model to ./cnn_model/MG_merge/tmp/word2vec.model.hdf5
25/25 [==============================] - 7s 117ms/step - loss: 5.1181 - mae: 1.5556 - val_loss: 0.7874 - val_mae: 0.7417
Epoch 2/100
25/25 [==============================] - ETA: 0s - loss: 0.7248 - mae: 0.5276
Epoch 2: val_loss improved from 0.78736 to 0.57421, saving model to ./cnn_model/MG_merge/tmp/word2vec.model.hdf5
25/25 [==============================] - 2s 98ms/step - loss: 0.7248 - mae: 0.5276 - val_loss: 0.5742 - val_mae: 0.4749
Epoch 3/100
25/25 [==============================] - ETA: 0s - loss: 0.5708 - mae: 0.4281
Epoch 3: val_loss improved from 0.57421 to 0.46686, saving model to ./cnn_model/MG_merge/tmp/word2vec.model.hdf5
25/25 [==============================] - 2s 97ms/step - loss: 0.5708 - mae: 0.4281 - val_loss: 0.4669 - val_mae: 0.2911


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:689: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


[0.7898564382883024, 0.895401236480248, 0.8945921456551583, 0.8775021410006084, -15.040280030095737, 0.8870295410026343, -15.04192970700353, -15.04001659674605, 0.606824009099239, 0.8827780960464037, -15.040280030095737, -15.040040183923402, 0.8532197817438554, 0.5474893126544897, -15.055156886640493, -15.040831191761544, -15.04006260303602, -15.04151861559031, -15.040613770375487, 0.8692060884236601]


In [ ]:
r2_SG_merge

[0.8404397759662624,
 0.8403280665778942,
 0.8355004636743163,
 0.8208336965200509,
 0.8426558181999703,
 0.8380181065462868,
 0.8445559758494802,
 0.8451903626131092,
 0.8014731286273536,
 0.8316815533481572,
 0.8318390389158271,
 0.8509652318431318,
 0.8252615713353465,
 0.8370728042445453,
 0.798863135421634,
 0.8065425692152892,
 0.8532220419550329,
 0.8381169469272299,
 0.8087596296790454,
 0.8464527605056376]

In [ ]:
r2_SG_split

[0.015441060128791428,
 0.06660595280856729,
 0.057687555104453,
 0.024239409486910612,
 -0.02467382782391736,
 0.15113259425115744,
 0.012020036471809936,
 0.18196206547355598,
 0.04624251756716846,
 0.017468553496777006,
 0.04855712997036077,
 0.17769785502510116,
 0.10004279282495943,
 0.009408563706876372,
 0.01185616168204262,
 -0.020606800611077558,
 0.1630530721057133,
 0.11683536432251918,
 0.09647933283335941,
 0.061938110133189395]

In [ ]:
r2_MG_merge

[0.7898564382883024,
 0.895401236480248,
 0.8945921456551583,
 0.8775021410006084,
 -15.040280030095737,
 0.8870295410026343,
 -15.04192970700353,
 -15.04001659674605,
 0.606824009099239,
 0.8827780960464037,
 -15.040280030095737,
 -15.040040183923402,
 0.8532197817438554,
 0.5474893126544897,
 -15.055156886640493,
 -15.040831191761544,
 -15.04006260303602,
 -15.04151861559031,
 -15.040613770375487,
 0.8692060884236601]

### **1.2.1原卷积神经网络模型预测(2)**

In [ ]:
path

'weighted_11:26_Falsedays_Trueinit_Truetrainable_0.010000dpt_0.001000lr_3fz_100fn_17maxlen_900dim_Truefilter_Truetransform 128batch_10window'

In [48]:
r2_SG_merge=[]
seed = 1234

def test_single_channel_merge_model(model, modelpath,X_test,demographics_test,y_test, index):
    print('Testing model...') 
    model.load_weights("/content/gdrive/MyDrive/cnn_model/%s.hdf5"%(modelpath))
    y_pred = model.predict([X_test, demographics_test], batch_size=args.batchsize, verbose=1)
    r2,rmse = evaluation(y_test, y_pred)
    print(r2)
    r2_SG_merge.append(r2)
    name = 'SG_merge%d'%(index)
    if args.isdays:
        daysplots(y_test, y_pred, r2, rmse, name)
    else:
        costplots(y_test, y_pred, r2, rmse, name)

r2_SG_split=[]
def test_single_channel_split_model(model, modelpath, X1_test,  demographics_test, y_test,index):
    print('Testing model...') 
    model.load_weights("/content/gdrive/MyDrive/cnn_model/%s.hdf5"%(modelpath))
    y_pred = model.predict([X1_test,  demographics_test], batch_size=args.batchsize, verbose=1)
    r2,rmse = evaluation(y_test, y_pred)
    r2_SG_split.append(r2)
    name = 'SG_merge%d'%(index)
    if args.isdays:
        daysplots(y_test, y_pred, r2, rmse, name)
    else:
        costplots(y_test, y_pred, r2, rmse, name)

r2_MG_merge=[]
def test_multi_channel_split_model(model, modelpath, X1_test,  demographics_test, y_test,index):
    print('Testing model...') 
    model.load_weights("/content/gdrive/MyDrive/cnn_model/%s.hdf5"%(modelpath))
    y_pred = model.predict([X1_test, demographics_test], batch_size=args.batchsize, verbose=1)
    print("y_pred:{}".format(y_pred.shape))
    r2,rmse = evaluation(y_test, y_pred)
    r2_MG_merge.append(r2)
    name = 'MG_split%d'%(index)
    if args.isdays:
        daysplots(y_test, y_pred, r2, rmse, name)
    else:
        costplots(y_test, y_pred, r2, rmse, name)

for i in range(1):
  modelpath = 'SG_merge' + path
  model = single_channel_merge_model(demgras_dim)
  if not args.test:
      train_single_channel_merge_model(model, modelpath, X_train, demographics_train, y_train)
  print('testing on the testing datasets.....')
  print("modelpath:",modelpath)
  test_single_channel_merge_model(model, modelpath, X_test, demographics_test, y_test, 3)
  print('testing on the filtered testing datasets.....')
  print(r2_SG_merge)


for i in range(1):
  modelpath = 'SG_split' + path
  model = single_channel_split_model(demgras_dim)
  if not args.test:
      train_single_channel_split_model(model, modelpath, X1_train, demographics_train, y_train)
  print(f,'testing on the testing datasets.....')
  test_single_channel_split_model(model, modelpath, X1_test,  demographics_test,y_test,3)
  print('testing on the filtered testing datasets.....')
  print(r2_SG_split)


for i in range(1):
  modelpath = 'MG_merge' + path
  model= multi_channel_split_model(demgras_dim)
  if not args.test:
      train_multi_channel_split_model(model, modelpath, X1_train, demographics_train, y_train)
  extract_patientvec(model, modelpath, disease, demographics)
  print(f,'testing on the testing datasets.....')
  test_multi_channel_split_model(model, modelpath, X1_test, demographics_test,y_test,3)
  print(r2_MG_merge)

开始模型1:单尺度融合模型
initialize embedding layer with pre-training vectors
依据预训练向量 初始化嵌入层
embedding layers trainalble True
添加人口统计信息，融合
Model: "model_43"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_27 (InputLayer)          [(None, 17)]         0           []                               
                                                                                                  
 embedding_31 (Embedding)       (None, 17, 900)      649800      ['input_27[0][0]']               
                                                                                                  
 conv1d_39 (Conv1D)             (None, 17, 100)      270100      ['embedding_31[0][0]']           
                                                                                                  
 conv1d_40 (Conv1D)             (None, 17, 100)      270100    

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


 99/100 [============================>.] - ETA: 0s - loss: 5.9327 - mae: 1.6349
Epoch 1: val_loss improved from inf to 0.84720, saving model to /content/gdrive/MyDrive/cnn_model/SG_mergeweighted_53:47_Falsedays_Trueinit_Truetrainable_0.010000dpt_0.001000lr_3fz_100fn_17maxlen_900dim_Truefilter_Truetransform 128batch_10window.hdf5
100/100 [==============================] - 2s 12ms/step - loss: 5.9214 - mae: 1.6328 - val_loss: 0.8472 - val_mae: 0.6172
Epoch 2/200
 95/100 [===========================>..] - ETA: 0s - loss: 0.8654 - mae: 0.6256
Epoch 2: val_loss improved from 0.84720 to 0.75428, saving model to /content/gdrive/MyDrive/cnn_model/SG_mergeweighted_53:47_Falsedays_Trueinit_Truetrainable_0.010000dpt_0.001000lr_3fz_100fn_17maxlen_900dim_Truefilter_Truetransform 128batch_10window.hdf5
100/100 [==============================] - 1s 11ms/step - loss: 0.8644 - mae: 0.6249 - val_loss: 0.7543 - val_mae: 0.5834
Epoch 3/200
 96/100 [===========================>..] - ETA: 0s - loss: 0.7836 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:446: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:457: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


testing on the filtered testing datasets.....
[0.866782333186304]
initialize embedding layer with pre-training vectors
依据预训练向量，初始化嵌入层...
embedding layers trainalble True
Model: "model_44"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_29 (InputLayer)          [(None, 17)]         0           []                               
                                                                                                  
 embedding_32 (Embedding)       (None, 17, 900)      649800      ['input_29[0][0]']               
                                                                                                  
 conv1d_42 (Conv1D)             (None, 17, 100)      270100      ['embedding_32[0][0]']           
                                                                                                  
 conv1d_43 (Conv1D) 

...

testing on the filtered testing datasets.....
[0.142092508980323]
demgras_dim:10
demgras_in.shape:(None, 10)
Model: "model_45"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_31 (InputLayer)          [(None, 17)]         0           []                               
                                                                                                  
 embedding_33 (Embedding)       (None, 17, 900)      649800      ['input_31[0][0]']               
                                                                                                  
 embedding_34 (Embedding)       (None, 17, 900)      649800      ['input_31[0][0]']               
                                                                                                  
 embedding_35 (Embedding)       (None, 17, 900)      649800      ['input_31[0][0]

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


None
Training the merge model....
Epoch 1/200
 99/100 [============================>.] - ETA: 0s - loss: 2.0968 - mae: 0.8645
Epoch 1: val_loss improved from inf to 0.66916, saving model to /content/gdrive/MyDrive/cnn_model/MG_mergeweighted_53:47_Falsedays_Trueinit_Truetrainable_0.010000dpt_0.001000lr_3fz_100fn_17maxlen_900dim_Truefilter_Truetransform 128batch_10window.hdf5
100/100 [==============================] - 5s 35ms/step - loss: 2.0939 - mae: 0.8643 - val_loss: 0.6692 - val_mae: 0.7182
Epoch 2/200
 98/100 [============================>.] - ETA: 0s - loss: 0.9356 - mae: 0.5546
Epoch 2: val_loss improved from 0.66916 to 0.43464, saving model to /content/gdrive/MyDrive/cnn_model/MG_mergeweighted_53:47_Falsedays_Trueinit_Truetrainable_0.010000dpt_0.001000lr_3fz_100fn_17maxlen_900dim_Truefilter_Truetransform 128batch_10window.hdf5
100/100 [==============================] - 3s 31ms/step - loss: 0.9452 - mae: 0.5570 - val_loss: 0.4346 - val_mae: 0.3450
Epoch 3/200
 99/100 [===========

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:446: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:457: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:468: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warnin

[0.8993649768118412]


In [49]:
r2_SG_merge

[0.866782333186304]

In [50]:
r2_SG_split

[0.142092508980323]

In [51]:
r2_MG_merge

[0.8993649768118412]

In [34]:
for i in range(1):
  modelpath = 'SG_merge' + path
  model = single_channel_merge_model(demgras_dim)
  if not args.test:
      train_single_channel_merge_model(model, modelpath, X_train, demographics_train, y_train)
  print('testing on the testing datasets.....')
  print("modelpath:",modelpath)
  test_single_channel_merge_model(model, modelpath, X_test, demographics_test, y_test, 3)
  print('testing on the filtered testing datasets.....')
  print(r2_SG_merge)

开始模型1:单尺度融合模型
initialize embedding layer with pre-training vectors
依据预训练向量 初始化嵌入层
embedding layers trainalble True
添加人口统计信息，融合
Model: "model_36"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_23 (InputLayer)          [(None, 17)]         0           []                               
                                                                                                  
 embedding_26 (Embedding)       (None, 17, 900)      649800      ['input_23[0][0]']               
                                                                                                  
 conv1d_33 (Conv1D)             (None, 17, 100)      270100      ['embedding_26[0][0]']           
                                                                                                  
 conv1d_34 (Conv1D)             (None, 17, 100)      270100    

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


None
Training the merge model....
Epoch 1/200
100/100 [==============================] - ETA: 0s - loss: 5.0304 - mae: 1.4754
Epoch 1: val_loss improved from inf to 0.90908, saving model to /content/gdrive/MyDrive/cnn_model/SG_mergeweighted_7:35_Falsedays_Trueinit_Truetrainable_0.010000dpt_0.001000lr_3fz_100fn_17maxlen_900dim_Truefilter_Truetransform 128batch_10window.hdf5
100/100 [==============================] - 5s 21ms/step - loss: 5.0304 - mae: 1.4754 - val_loss: 0.9091 - val_mae: 0.6343
Epoch 2/200
 98/100 [============================>.] - ETA: 0s - loss: 0.8991 - mae: 0.6286
Epoch 2: val_loss improved from 0.90908 to 0.81544, saving model to /content/gdrive/MyDrive/cnn_model/SG_mergeweighted_7:35_Falsedays_Trueinit_Truetrainable_0.010000dpt_0.001000lr_3fz_100fn_17maxlen_900dim_Truefilter_Truetransform 128batch_10window.hdf5
100/100 [==============================] - 2s 16ms/step - loss: 0.8980 - mae: 0.6282 - val_loss: 0.8154 - val_mae: 0.5878
Epoch 3/200
 96/100 [=============

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:446: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:457: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


testing on the filtered testing datasets.....
[0.8562258751044397]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:468: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


## **1.3 机器学习预测——独热编码、多尺度编码**

### **1.3.1 导入模型**

In [ ]:
#===============================第一部分 独热编码 ============================================
from sklearn import tree
from sklearn import linear_model
from sklearn import svm
from sklearn import neighbors
from sklearn import ensemble
from sklearn import ensemble
from sklearn import ensemble
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import ExtraTreeRegressor
import xgboost as xgb
def onehot_RF(X_train, X_test, y_train,y_test, demographics_train, demographics_test):
    #one_hot_encoder
    print('RandomForestRegressor with one_hot encoding...') 
    onehot_train = one_hot_encoder(X_train, nb_words)
    #train_mat = onehot_train
    train_mat = np.hstack((onehot_train, demographics_train))#merge the demographics info
    rf = ensemble.RandomForestRegressor(n_estimators=10, n_jobs=-1)
    rf.fit(train_mat, y_train.ravel())
    #test_mat = onehot_test
    onehot_test = one_hot_encoder(X_test, nb_words)
    test_mat = np.hstack((onehot_test, demographics_test))#merge the demographics info
    y_pred = rf.predict(test_mat)
    r2, rmse =  evaluation(y_test, y_pred)
    print('testing on filter samples...')

def onehot_SVR(X_train, X_test, y_train,y_test, demographics_train, demographics_test):
    print('SVR with one_hot encoding...') 
    onehot_train = one_hot_encoder(X_train, nb_words)
    #train_mat = onehot_train
    train_mat = np.hstack((onehot_train, demographics_train))#merge the demographics info
    rf = svm.SVR()
    rf.fit(train_mat, y_train.ravel())
    #test_mat = onehot_test
    onehot_test = one_hot_encoder(X_test, nb_words)
    test_mat = np.hstack((onehot_test, demographics_test))#merge the demographics info
    y_pred = rf.predict(test_mat)
    r2, rmse =  evaluation(y_test, y_pred)
    print('testing on filter samples...') 

def onehot_AdaBoost(X_train, X_test, y_train,y_test, demographics_train, demographics_test):
    print('AdaBoost with one_hot encoding...') 
    onehot_train = one_hot_encoder(X_train, nb_words)
    #train_mat = onehot_train
    train_mat = np.hstack((onehot_train, demographics_train))#merge the demographics info
    rf = ensemble.AdaBoostRegressor(n_estimators=10,random_state=90)
    rf.fit(train_mat, y_train.ravel())
    #test_mat = onehot_test
    onehot_test = one_hot_encoder(X_test, nb_words)
    test_mat = np.hstack((onehot_test, demographics_test))#merge the demographics info
    y_pred = rf.predict(test_mat)
    r2, rmse =  evaluation(y_test, y_pred)
    print('testing on filter samples...')

def onehot_GradientBoosting(X_train, X_test, y_train,y_test, demographics_train, demographics_test):
    print('GradientBoosting with one_hot encoding...') 
    onehot_train = one_hot_encoder(X_train, nb_words)
    #train_mat = onehot_train
    train_mat = np.hstack((onehot_train, demographics_train))#merge the demographics info
    rf = ensemble.GradientBoostingRegressor(n_estimators=10,random_state=90)
    rf.fit(train_mat, y_train.ravel())
    #test_mat = onehot_test
    onehot_test = one_hot_encoder(X_test, nb_words)
    test_mat = np.hstack((onehot_test, demographics_test))#merge the demographics info
    y_pred = rf.predict(test_mat)
    r2, rmse =  evaluation(y_test, y_pred)
    print('testing on filter samples...')

def onehot_Bagging(X_train, X_test, y_train,y_test, demographics_train, demographics_test):
    print('Bagging with one_hot encoding...') 
    onehot_train = one_hot_encoder(X_train, nb_words)
    #train_mat = onehot_train
    train_mat = np.hstack((onehot_train, demographics_train))#merge the demographics info
    rf = BaggingRegressor(n_estimators=10,random_state=90)
    rf.fit(train_mat, y_train.ravel())
    #test_mat = onehot_test
    onehot_test = one_hot_encoder(X_test, nb_words)
    test_mat = np.hstack((onehot_test, demographics_test))#merge the demographics info
    y_pred = rf.predict(test_mat)
    r2, rmse =  evaluation(y_test, y_pred)
    print('testing on filter samples...')

def onehot_ExtraTree(X_train, X_test, y_train,y_test, demographics_train, demographics_test):
    print('ExtraTree with one_hot encoding...') 
    onehot_train = one_hot_encoder(X_train, nb_words)
    #train_mat = onehot_train
    train_mat = np.hstack((onehot_train, demographics_train))#merge the demographics info
    rf = ExtraTreeRegressor(random_state=90)
    rf.fit(train_mat, y_train.ravel())
    #test_mat = onehot_test
    onehot_test = one_hot_encoder(X_test, nb_words)
    test_mat = np.hstack((onehot_test, demographics_test))#merge the demographics info
    y_pred = rf.predict(test_mat)
    r2, rmse =  evaluation(y_test, y_pred)
    print('testing on filter samples...')

def onehot_DecisionTree(X_train, X_test, y_train,y_test, demographics_train, demographics_test):
    print('DecisionTree with one_hot encoding...') 
    onehot_train = one_hot_encoder(X_train, nb_words)
    #train_mat = onehot_train
    train_mat = np.hstack((onehot_train, demographics_train))#merge the demographics info
    rf = tree.DecisionTreeRegressor(random_state=90)
    rf.fit(train_mat, y_train.ravel())
    #test_mat = onehot_test
    onehot_test = one_hot_encoder(X_test, nb_words)
    test_mat = np.hstack((onehot_test, demographics_test))#merge the demographics info
    y_pred = rf.predict(test_mat)
    r2, rmse =  evaluation(y_test, y_pred)
    print('testing on filter samples...')

def onehot_KNeighbors(X_train, X_test, y_train,y_test, demographics_train, demographics_test):
    print('KNeighbors with one_hot encoding...') 
    onehot_train = one_hot_encoder(X_train, nb_words)
    #train_mat = onehot_train
    train_mat = np.hstack((onehot_train, demographics_train))#merge the demographics info
    rf = neighbors.KNeighborsRegressor()
    rf.fit(train_mat, y_train.ravel())
    #test_mat = onehot_test
    onehot_test = one_hot_encoder(X_test, nb_words)
    test_mat = np.hstack((onehot_test, demographics_test))#merge the demographics info
    y_pred = rf.predict(test_mat)
    r2, rmse =  evaluation(y_test, y_pred)
    print('testing on filter samples...')

def onehot_XGBoost(X_train, X_test, y_train,y_test, demographics_train, demographics_test):
    print('XGBoost with one_hot encoding...') 
    onehot_train = one_hot_encoder(X_train, nb_words)
    #train_mat = onehot_train
    train_mat = np.hstack((onehot_train, demographics_train))#merge the demographics info
    rf = xgb.XGBRegressor(n_estimators=10,random_state=90)
    rf.fit(train_mat, y_train.ravel())
    #test_mat = onehot_test
    onehot_test = one_hot_encoder(X_test, nb_words)
    test_mat = np.hstack((onehot_test, demographics_test))#merge the demographics info
    y_pred = rf.predict(test_mat)
    r2, rmse =  evaluation(y_test, y_pred)
    print('testing on filter samples...')

def onehot_LR(X_train, X_test, y_train,y_test, demographics_train, demographics_test):
    #one_hot_encoder
    print('LR with one_hot encoding...') 
    onehot_train = one_hot_encoder(X_train, nb_words)
    #train_mat = onehot_train
    train_mat = np.hstack((onehot_train, demographics_train))#merge the demographics info
    #rf = ensemble.RandomForestRegressor(n_estimators=10, n_jobs=-1)
    #rf.fit(train_mat, y_train.ravel())
    rf = LinearRegression(normalize=True, n_jobs=-1)
    rf.fit(train_mat, y_train.ravel())
    #test_mat = onehot_test
    onehot_test = one_hot_encoder(X_test, nb_words)
    test_mat = np.hstack((onehot_test, demographics_test))#merge the demographics info
    y_pred = rf.predict(test_mat)
    r2, rmse =  evaluation(y_test, y_pred)
    print('testing on filter samples...') 

#============================== 第二部分 预测函数 单尺度、多尺度======================================================



def embedding_RF(X_train, X_test,y_train,y_test,demographics_train,demographics_test):
    #word2vec encoder
    print(f,'RandomForestRegressor with embedding encoding...')
    #train_mat = embedding_encoder(X_train, index_embedding)
    #test_mat = embedding_encoder(X_test, index_embedding)
    #train_mat = np.hstack((train_mat, demographics_train))#merge the demographics info

    #rf = ensemble.RandomForestRegressor(n_estimators=10,random_state=90)
    #rf.fit(train_mat, y_train.ravel())
    #test_mat = np.hstack((test_mat, demographics_test))#merge the demographics info
    #y_pred = rf.predict(test_mat)
    #r2, rmse = evaluation(y_test, y_pred)
    
    #multichannel word2vec embedding encoder
    print(f,'RandomForestRegressor with multichannel embedding encoding...')
    train_mat = multichannel_embedding_encoder(X_train, index_embedding, index_embedding1, index_embedding2, index_embedding3)
    test_mat = multichannel_embedding_encoder(X_test, index_embedding, index_embedding1, index_embedding2, index_embedding3)
    train_mat = np.hstack((train_mat, demographics_train))#merge the demographics info
    rf = ensemble.RandomForestRegressor(n_estimators=10,random_state=90) #n_jobs=-1便是使用全部的CPU
    rf.fit(train_mat, y_train.ravel())
    test_mat = np.hstack((test_mat, demographics_test))#merge the demographics info
    y_pred = rf.predict(test_mat)
    r2, rmse = evaluation(y_test, y_pred)
    index = 1
    name = 'MG_RF%d'%(index)
    if args.isdays:
        daysplots(y_test, y_pred, r2, rmse, name)
    else:
        costplots(y_test, y_pred, r2, rmse, name)
    print('testing on filter testing samples...')

    index = 2
    name = 'MG_RF%d'%(index)
    if args.isdays:
        daysplots(y_test, y_pred, r2, rmse, name)
    else:
        costplots(y_test, y_pred, r2, rmse, name)
    print('testing on filter samples...')

    index = 3
    name = 'MG_RF%d'%(index)
    if args.isdays:
        daysplots(y_test, y_pred, r2, rmse, name)
    else:
        costplots(y_test, y_pred, r2, rmse, name)


def embedding_LR(X_train, X_test,y_train,y_test,demographics_train,demographics_test):
    #word2vec encoder
    print(f,'Linear regression with embedding encoding...')
    #train_mat = embedding_encoder(X_train, index_embedding)
    #test_mat = embedding_encoder(X_test, index_embedding)
    #train_mat = np.hstack((train_mat, demographics_train))#merge the demographics info
    #print('LR_train_mat')
    #print(train_mat.shape)
    #lr = LinearRegression(normalize=True)
    #lr.fit(train_mat, y_train.ravel())
    #test_mat = np.hstack((test_mat, demographics_test))#merge the demographics info
    #y_pred = lr.predict(test_mat)
    #r2, rmse = evaluation(y_test, y_pred)

    #multichannel word2vec embedding encoder
    print(f,'Linear regression with multichannel embedding encoding...')
    train_mat = multichannel_embedding_encoder(X_train, index_embedding, index_embedding1, index_embedding2, index_embedding3)
    test_mat = multichannel_embedding_encoder(X_test, index_embedding, index_embedding1, index_embedding2, index_embedding3)
    train_mat = np.hstack((train_mat, demographics_train))#merge the demographics info
    lr = svm.SVR()
    lr.fit(train_mat, y_train.ravel())
    test_mat = np.hstack((test_mat, demographics_test))#merge the demographics info
    y_pred = lr.predict(test_mat)
    r2, rmse = evaluation(y_test, y_pred)
    

def embedding_SVR(X_train, X_test,y_train,y_test,demographics_train,demographics_test):
    #1、word2vec encoder
    print(f,'SVR with embedding encoding...')
    #train_mat = embedding_encoder(X_train, index_embedding)
    #test_mat = embedding_encoder(X_test, index_embedding)
    #train_mat = np.hstack((train_mat, demographics_train))#merge the demographics info
    #print('SVR_train_mat')
    #lr = svm.SVR()
    #lr.fit(train_mat, y_train.ravel())
    #test_mat = np.hstack((test_mat, demographics_test))#merge the demographics info
    #y_pred = lr.predict(test_mat)
    #r2, rmse = evaluation(y_test, y_pred)
    #2、multichannel word2vec embedding encoder
    print(f,'SVR with multichannel embedding encoding...')
    train_mat = multichannel_embedding_encoder(X_train, index_embedding, index_embedding1, index_embedding2, index_embedding3)
    test_mat = multichannel_embedding_encoder(X_test, index_embedding, index_embedding1, index_embedding2, index_embedding3)
    train_mat = np.hstack((train_mat, demographics_train))#merge the demographics info
    lr = svm.SVR()
    lr.fit(train_mat, y_train.ravel())
    test_mat = np.hstack((test_mat, demographics_test))#merge the demographics info
    y_pred = lr.predict(test_mat)
    r2, rmse = evaluation(y_test, y_pred)

def embedding_AdaBoost(X_train, X_test,y_train,y_test,demographics_train,demographics_test):
    #1、word2vec encoder
    print(f,'AdaBoost with embedding encoding...')
    #train_mat = embedding_encoder(X_train, index_embedding)
    #test_mat = embedding_encoder(X_test, index_embedding)
    #train_mat = np.hstack((train_mat, demographics_train))#merge the demographics info
    #lr = ensemble.AdaBoostRegressor(n_estimators=10,random_state=90)
    #lr.fit(train_mat, y_train.ravel())
    #test_mat = np.hstack((test_mat, demographics_test))#merge the demographics info
    #y_pred = lr.predict(test_mat)
    #r2, rmse = evaluation(y_test, y_pred)
    #2、multichannel word2vec embedding encoder
    print(f,'AdaBoost with multichannel embedding encoding...')
    train_mat = multichannel_embedding_encoder(X_train, index_embedding, index_embedding1, index_embedding2, index_embedding3)
    test_mat = multichannel_embedding_encoder(X_test, index_embedding, index_embedding1, index_embedding2, index_embedding3)
    train_mat = np.hstack((train_mat, demographics_train))#merge the demographics info
    lr = ensemble.AdaBoostRegressor(n_estimators=10,random_state=90)
    lr.fit(train_mat, y_train.ravel())
    test_mat = np.hstack((test_mat, demographics_test))#merge the demographics info
    y_pred = lr.predict(test_mat)
    r2, rmse = evaluation(y_test, y_pred)

def embedding_GradientBoosting(X_train, X_test,y_train,y_test,demographics_train,demographics_test):
    #1、word2vec encoder
    #print(f,'GradientBoosting with embedding encoding...')
    #train_mat = embedding_encoder(X_train, index_embedding)
    #test_mat = embedding_encoder(X_test, index_embedding)
    #train_mat = np.hstack((train_mat, demographics_train))#merge the demographics info
    #lr = ensemble.GradientBoostingRegressor(n_estimators=10,random_state=90)
    #lr.fit(train_mat, y_train.ravel())
    #test_mat = np.hstack((test_mat, demographics_test))#merge the demographics info
    #y_pred = lr.predict(test_mat)
    #r2, rmse = evaluation(y_test, y_pred)
    #2、multichannel word2vec embedding encoder
    print(f,'GradientBoosting with multichannel embedding encoding...')
    train_mat = multichannel_embedding_encoder(X_train, index_embedding, index_embedding1, index_embedding2, index_embedding3)
    test_mat = multichannel_embedding_encoder(X_test, index_embedding, index_embedding1, index_embedding2, index_embedding3)
    train_mat = np.hstack((train_mat, demographics_train))#merge the demographics info
    lr = ensemble.GradientBoostingRegressor(n_estimators=10,random_state=90)
    lr.fit(train_mat, y_train.ravel())
    test_mat = np.hstack((test_mat, demographics_test))#merge the demographics info
    y_pred = lr.predict(test_mat)
    r2, rmse = evaluation(y_test, y_pred)


def embedding_Bagging(X_train, X_test,y_train,y_test,demographics_train,demographics_test):
    #1、word2vec encoder
    print(f,'Bagging with embedding encoding...')
    #train_mat = embedding_encoder(X_train, index_embedding)
    #test_mat = embedding_encoder(X_test, index_embedding)
    #train_mat = np.hstack((train_mat, demographics_train))#merge the demographics info
    #lr = BaggingRegressor(n_estimators=10,random_state=90)
    #lr.fit(train_mat, y_train.ravel())
    #test_mat = np.hstack((test_mat, demographics_test))#merge the demographics info
    #y_pred = lr.predict(test_mat)
    #r2, rmse = evaluation(y_test, y_pred)
    #2、multichannel word2vec embedding encoder
    print(f,'Bagging with multichannel embedding encoding...')
    train_mat = multichannel_embedding_encoder(X_train, index_embedding, index_embedding1, index_embedding2, index_embedding3)
    test_mat = multichannel_embedding_encoder(X_test, index_embedding, index_embedding1, index_embedding2, index_embedding3)
    train_mat = np.hstack((train_mat, demographics_train))#merge the demographics info
    lr = BaggingRegressor(n_estimators=10,random_state=90)
    lr.fit(train_mat, y_train.ravel())
    test_mat = np.hstack((test_mat, demographics_test))#merge the demographics info
    y_pred = lr.predict(test_mat)
    r2, rmse = evaluation(y_test, y_pred)


def embedding_ExtraTree(X_train, X_test,y_train,y_test,demographics_train,demographics_test):
    #1、word2vec encoder
    print(f,'ExtraTree with embedding encoding...')
    #train_mat = embedding_encoder(X_train, index_embedding)
    #test_mat = embedding_encoder(X_test, index_embedding)
    #train_mat = np.hstack((train_mat, demographics_train))#merge the demographics info
    #lr = ExtraTreeRegressor(random_state=90)
    #lr.fit(train_mat, y_train.ravel())
    #test_mat = np.hstack((test_mat, demographics_test))#merge the demographics info
    #y_pred = lr.predict(test_mat)
    #r2, rmse = evaluation(y_test, y_pred)
    #2、multichannel word2vec embedding encoder
    print(f,'ExtraTree with multichannel embedding encoding...')
    train_mat = multichannel_embedding_encoder(X_train, index_embedding, index_embedding1, index_embedding2, index_embedding3)
    test_mat = multichannel_embedding_encoder(X_test, index_embedding, index_embedding1, index_embedding2, index_embedding3)
    train_mat = np.hstack((train_mat, demographics_train))#merge the demographics info
    lr = ExtraTreeRegressor(random_state=90)
    lr.fit(train_mat, y_train.ravel())
    test_mat = np.hstack((test_mat, demographics_test))#merge the demographics info
    y_pred = lr.predict(test_mat)
    r2, rmse = evaluation(y_test, y_pred)


def embedding_DecisionTree(X_train, X_test,y_train,y_test,demographics_train,demographics_test):
    #1、word2vec encoder
    print(f,'DecisionTree with embedding encoding...')
    #train_mat = embedding_encoder(X_train, index_embedding)
    #test_mat = embedding_encoder(X_test, index_embedding)
    #train_mat = np.hstack((train_mat, demographics_train))#merge the demographics info
    #lr = tree.DecisionTreeRegressor(random_state=90)
    #lr.fit(train_mat, y_train.ravel())
    #test_mat = np.hstack((test_mat, demographics_test))#merge the demographics info
    #y_pred = lr.predict(test_mat)
    #r2, rmse = evaluation(y_test, y_pred)
    #2、multichannel word2vec embedding encoder
    print(f,'DecisionTree with multichannel embedding encoding...')
    train_mat = multichannel_embedding_encoder(X_train, index_embedding, index_embedding1, index_embedding2, index_embedding3)
    test_mat = multichannel_embedding_encoder(X_test, index_embedding, index_embedding1, index_embedding2, index_embedding3)
    train_mat = np.hstack((train_mat, demographics_train))#merge the demographics info
    lr = tree.DecisionTreeRegressor(random_state=90)
    lr.fit(train_mat, y_train.ravel())
    test_mat = np.hstack((test_mat, demographics_test))#merge the demographics info
    y_pred = lr.predict(test_mat)
    r2, rmse = evaluation(y_test, y_pred)


def embedding_KNeighbors(X_train, X_test,y_train,y_test,demographics_train,demographics_test):
    #1、word2vec encoder
    print(f,'KNeighbors with embedding encoding...')
    #train_mat = embedding_encoder(X_train, index_embedding)
    #test_mat = embedding_encoder(X_test, index_embedding)
    #train_mat = np.hstack((train_mat, demographics_train))#merge the demographics info
    #lr = neighbors.KNeighborsRegressor()
    #lr.fit(train_mat, y_train.ravel())
    #test_mat = np.hstack((test_mat, demographics_test))#merge the demographics info
    #y_pred = lr.predict(test_mat)
    #r2, rmse = evaluation(y_test, y_pred)
    #2、multichannel word2vec embedding encoder
    print(f,'KNeighbors with multichannel embedding encoding...')
    train_mat = multichannel_embedding_encoder(X_train, index_embedding, index_embedding1, index_embedding2, index_embedding3)
    test_mat = multichannel_embedding_encoder(X_test, index_embedding, index_embedding1, index_embedding2, index_embedding3)
    train_mat = np.hstack((train_mat, demographics_train))#merge the demographics info
    lr = neighbors.KNeighborsRegressor()
    lr.fit(train_mat, y_train.ravel())
    test_mat = np.hstack((test_mat, demographics_test))#merge the demographics info
    y_pred = lr.predict(test_mat)
    r2, rmse = evaluation(y_test, y_pred)

def embedding_XGBoost(X_train, X_test,y_train,y_test,demographics_train,demographics_test):
    #1、word2vec encoder
    print(f,'XGBoost with embedding encoding...')
    #train_mat = embedding_encoder(X_train, index_embedding)
    #test_mat = embedding_encoder(X_test, index_embedding)
    #train_mat = np.hstack((train_mat, demographics_train))#merge the demographics info
    #lr = xgb.XGBRegressor(n_estimators=10,random_state=90)
    #lr.fit(train_mat, y_train.ravel())
    #test_mat = np.hstack((test_mat, demographics_test))#merge the demographics info
    #y_pred = lr.predict(test_mat)
    #r2, rmse = evaluation(y_test, y_pred)
    #2、multichannel word2vec embedding encoder
    print(f,'XGBoost with multichannel embedding encoding...')
    train_mat = multichannel_embedding_encoder(X_train, index_embedding, index_embedding1, index_embedding2, index_embedding3)
    test_mat = multichannel_embedding_encoder(X_test, index_embedding, index_embedding1, index_embedding2, index_embedding3)
    train_mat = np.hstack((train_mat, demographics_train))#merge the demographics info
    lr = xgb.XGBRegressor(n_estimators=10,random_state=90)
    lr.fit(train_mat, y_train.ravel())
    test_mat = np.hstack((test_mat, demographics_test))#merge the demographics info
    y_pred = lr.predict(test_mat)
    r2, rmse = evaluation(y_test, y_pred)


### **1.3.2 机器学习：X_train**

In [ ]:
#onehot_SVR(X_train, X_test,y_train,y_test,demographics_train,demographics_test)
#onehot_AdaBoost(X_train, X_test,y_train,y_test,demographics_train,demographics_test)
#onehot_GradientBoosting(X_train, X_test,y_train,y_test,demographics_train,demographics_test)
#onehot_RF(X_train, X_test,y_train,y_test,demographics_train,demographics_test)
#onehot_Bagging(X_train, X_test,y_train,y_test,demographics_train,demographics_test)
#onehot_ExtraTree(X_train, X_test,y_train,y_test,demographics_train,demographics_test)
#onehot_DecisionTree(X_train, X_test,y_train,y_test,demographics_train,demographics_test)
#onehot_LR(X_train, X_test,y_train,y_test,demographics_train,demographics_test)
#onehot_KNeighbors(X_train, X_test,y_train,y_test,demographics_train,demographics_test)
#onehot_XGBoost(X_train, X_test,y_train,y_test,demographics_train,demographics_test)
embedding_RF(X_train, X_test,y_train,y_test,demographics_train,demographics_test)

embedding_SVR(X_train, X_test,y_train,y_test,demographics_train,demographics_test)
embedding_AdaBoost(X_train, X_test,y_train,y_test,demographics_train,demographics_test)
embedding_GradientBoosting(X_train, X_test,y_train,y_test,demographics_train,demographics_test)
embedding_Bagging(X_train, X_test,y_train,y_test,demographics_train,demographics_test)
embedding_ExtraTree(X_train, X_test,y_train,y_test,demographics_train,demographics_test)
embedding_DecisionTree(X_train, X_test,y_train,y_test,demographics_train,demographics_test)
embedding_LR(X_train, X_test,y_train,y_test,demographics_train,demographics_test)
embedding_KNeighbors(X_train, X_test,y_train,y_test,demographics_train,demographics_test)
embedding_XGBoost(X_train, X_test,y_train,y_test,demographics_train,demographics_test)

### **1.3.2 机器学习：X1_train**

In [ ]:
embedding_RF(X1_train, X1_test,y_train,y_test,demographics_train,demographics_test)

/content/gdrive/MyDrive/15684_new_2.csv RandomForestRegressor with embedding encoding...
/content/gdrive/MyDrive/15684_new_2.csv RandomForestRegressor with multichannel embedding encoding...
开始步骤52/3:开始多尺度编码...
开始步骤52/3:开始多尺度编码...
y_test:(1569, 1)
y_pred:(1569,)
float64
MSE:0.06514871970351838, RMSE:0.2552424723738555, MAE:0.1028683127363143, R2:0.9239845174521355
开始步骤92/2:预测可视化-医疗费用


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:409: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


testing on filter testing samples...
开始步骤92/2:预测可视化-医疗费用


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:409: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


testing on filter samples...
开始步骤92/2:预测可视化-医疗费用


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:409: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


In [ ]:
#onehot_SVR(X1_train, X1_test,y_train,y_test,demographics_train,demographics_test)
#onehot_AdaBoost(X1_train, X1_test,y_train,y_test,demographics_train,demographics_test)
#onehot_GradientBoosting(X1_train, X1_test,y_train,y_test,demographics_train,demographics_test)
#onehot_RF(X1_train, X1_test,y_train,y_test,demographics_train,demographics_test)
#onehot_Bagging(X1_train, X1_test,y_train,y_test,demographics_train,demographics_test)
#onehot_ExtraTree(X1_train, X1_test,y_train,y_test,demographics_train,demographics_test)
#onehot_DecisionTree(X1_train, X1_test,y_train,y_test,demographics_train,demographics_test)
#onehot_LR(X1_train, X1_test,y_train,y_test,demographics_train,demographics_test)
#onehot_KNeighbors(X1_train, X1_test,y_train,y_test,demographics_train,demographics_test)
#onehot_XGBoost(X1_train, X1_test,y_train,y_test,demographics_train,demographics_test)

embedding_SVR(X1_train, X1_test,y_train,y_test,demographics_train,demographics_test)
embedding_AdaBoost(X1_train, X1_test,y_train,y_test,demographics_train,demographics_test)
embedding_GradientBoosting(X1_train, X1_test,y_train,y_test,demographics_train,demographics_test)
embedding_Bagging(X1_train, X1_test,y_train,y_test,demographics_train,demographics_test)
embedding_ExtraTree(X1_train, X1_test,y_train,y_test,demographics_train,demographics_test)
embedding_DecisionTree(X1_train, X1_test,y_train,y_test,demographics_train,demographics_test)
embedding_LR(X1_train, X1_test,y_train,y_test,demographics_train,demographics_test)
embedding_KNeighbors(X1_train, X1_test,y_train,y_test,demographics_train,demographics_test)
embedding_XGBoost(X1_train, X1_test,y_train,y_test,demographics_train,demographics_test)
embedding_RF(X1_train, X1_test,y_train,y_test,demographics_train,demographics_test)

/content/gdrive/MyDrive/15684_new_2.csv SVR with embedding encoding...
/content/gdrive/MyDrive/15684_new_2.csv SVR with multichannel embedding encoding...
开始多尺度编码...
开始多尺度编码...


findfont: Font family ['Times New Roman'] not found. Falling back to DejaVu Sans.


y_test:(1569, 1)
y_pred:(1569,)
float64
MSE:0.8320842966076956, RMSE:0.9121865470438026, MAE:0.42257155299910787, R2:0.0291245996086944
/content/gdrive/MyDrive/15684_new_2.csv AdaBoost with embedding encoding...
/content/gdrive/MyDrive/15684_new_2.csv AdaBoost with multichannel embedding encoding...
开始多尺度编码...
开始多尺度编码...
y_test:(1569, 1)
y_pred:(1569,)
float64
MSE:0.0915339382750846, RMSE:0.3025457622824762, MAE:0.16853713599393558, R2:0.8931982620817147
/content/gdrive/MyDrive/15684_new_2.csv GradientBoosting with multichannel embedding encoding...
开始多尺度编码...
开始多尺度编码...
y_test:(1569, 1)
y_pred:(1569,)
float64
MSE:0.16220130679550598, RMSE:0.40274223368738715, MAE:0.25416818042611694, R2:0.8107436237877639
/content/gdrive/MyDrive/15684_new_2.csv Bagging with embedding encoding...
/content/gdrive/MyDrive/15684_new_2.csv Bagging with multichannel embedding encoding...
开始多尺度编码...
开始多尺度编码...
y_test:(1569, 1)
y_pred:(1569,)
float64
MSE:0.06940223087252957, RMSE:0.26344303155052246, MAE:0.10

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  FutureWarning,


y_test:(1569, 1)
y_pred:(1569,)
float64
MSE:5.3015374352163227e+23, RMSE:728116572755.7863, MAE:55331298742.74734, R2:-6.185830331240913e+23
/content/gdrive/MyDrive/15684_new_2.csv KNeighbors with embedding encoding...
/content/gdrive/MyDrive/15684_new_2.csv KNeighbors with multichannel embedding encoding...
开始多尺度编码...
开始多尺度编码...
y_test:(1569, 1)
y_pred:(1569,)
float32
MSE:0.4764047861099243, RMSE:0.690220832824707, MAE:0.29000937938690186, R2:0.4441311894182388
/content/gdrive/MyDrive/15684_new_2.csv XGBoost with embedding encoding...
/content/gdrive/MyDrive/15684_new_2.csv XGBoost with multichannel embedding encoding...
开始多尺度编码...
开始多尺度编码...
[11:23:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
y_test:(1569, 1)
y_pred:(1569,)
float32
MSE:3.898815393447876, RMSE:1.9745417833328247, MAE:1.9325222969055176, R2:-3.549135038679654
/content/gdrive/MyDrive/15684_new_2.csv RandomForestRegressor with embedding encoding..

# **2、RF.py**

X1_train_1,X1_val_1, demographics_train_1,demographics_val_1,y_train_1,y_val_1

=
train_test_split(X1_train, demographics_train, y_train, test_size=0.1, random_state=1) 


## **2.1 RF1: 特征数据**

### **2.1.1 数据整理**

In [ ]:
demographics_train_1.shape

(12703, 10)

In [ ]:
y_train_1

array([[5.71278  ],
       [5.70374  ],
       [6.3433576],
       ...,
       [5.70374  ],
       [5.70374  ],
       [5.70374  ]], dtype=float32)

In [ ]:
#y_val_1=np.log(y_val_1)
y_val_1

array([[5.70374 ],
       [5.70374 ],
       [5.70374 ],
       ...,
       [8.166428],
       [5.70374 ],
       [5.70374 ]], dtype=float32)

### 2.1.2 **数据预测：验证集**

In [41]:
from sklearn import ensemble
model_RandomForestRegressor = ensemble.RandomForestRegressor(random_state=90)
model_RandomForestRegressor.fit(demographics_train_1,y_train_1)
y_RF_hat = model_RandomForestRegressor.predict(demographics_val_1)
R2_RF_1=r2_score(y_val_1, y_RF_hat)
print(R2_RF_1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.8888829531653479


### **2.1.3 数据预测：测试集**

In [ ]:
model_RandomForestRegressor = ensemble.RandomForestRegressor(random_state=90)
model_RandomForestRegressor.fit(demographics_train,y_train)
y_RF_pred_1 = model_RandomForestRegressor.predict(demographics_test)
R2=r2_score(y_test, y_RF_pred_1)
print(R2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.8661664754472297


## **2.2 RF2: 疾病数据+特征数据**

In [ ]:
#可以试试这个数据集

train_mat = multichannel_embedding_encoder(X1_train_1, index_embedding, index_embedding1, index_embedding2, index_embedding3)
val_mat= multichannel_embedding_encoder(X1_val_1, index_embedding, index_embedding1, index_embedding2, index_embedding3)
test_mat = multichannel_embedding_encoder(X1_test, index_embedding, index_embedding1, index_embedding2, index_embedding3)

train_mat = np.hstack((train_mat, demographics_train_1))#训练集
val_mat = np.hstack((val_mat, demographics_val_1))#验证集
test_mat = np.hstack((test_mat, demographics_test))#测试集

#测试集预测 
train_mat_all=multichannel_embedding_encoder(X1_train, index_embedding, index_embedding1, index_embedding2, index_embedding3)
train_all = np.hstack((train_mat_all, demographics_train))#训练集+验证集

开始多尺度编码...
开始多尺度编码...
开始多尺度编码...
开始多尺度编码...


### **2.2.1 数据预测：验证集** 

In [ ]:
from sklearn import ensemble
model_RandomForestRegressor = ensemble.RandomForestRegressor(random_state=90)

data_train_1=np.concatenate((X1_train_1, demographics_train_1),axis=1)
#data_train_1=train_mat
model_RandomForestRegressor.fit(data_train_1,y_train_1)


data_val_1=np.concatenate((X1_val_1, demographics_val_1),axis=1)
#data_val_1=val_mat
y_RF_hat = model_RandomForestRegressor.predict(data_val_1)

R2_RF_2=r2_score(y_val_1, y_RF_hat)
print(R2_RF_2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.9392796414986722


### **2.2.2 数据预测：测试集**

In [ ]:
y_train

array([[9.021788 ],
       [5.297479 ],
       [6.9925523],
       ...,
       [5.70374  ],
       [7.8099203],
       [5.297479 ]], dtype=float32)

In [ ]:
model_RandomForestRegressor = ensemble.RandomForestRegressor(random_state=90)
data_train=np.concatenate((X1_train, demographics_train),axis=1)
#data_train=train_mat_all
model_RandomForestRegressor.fit(data_train,y_train) #数据拟合
data_test=np.concatenate((X1_test, demographics_test),axis=1)
#data_test=test_mat
y_RF_pred_2 = model_RandomForestRegressor.predict(data_test) #模型预测
R2=r2_score(y_test, y_RF_pred_2) #拟合优度
print(R2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


0.9128287590325465


## **2.3 RF3（原代码）：X1_train**

### **2.3.1 独热编码、单尺度——随机森林模型**

In [ ]:
comm_X1111=pd.DataFrame(comm_X1) #
comm_X1111.to_csv('/content/gdrive/MyDrive/comm_X1.csv')

In [47]:
#one_hot_encoder 独热编码
print('RandomForestRegressor with one_hot encoding...') 
onehot_train = one_hot_encoder(X1_train, nb_words)
#train_mat = onehot_train
train_mat = np.hstack((onehot_train, demographics_train))#merge the demographics info
rf = ensemble.RandomForestRegressor(n_estimators=10, n_jobs=-1)
rf.fit(train_mat, y_train.ravel())
#test_mat = onehot_test
onehot_test = one_hot_encoder(X1_test, nb_words)
test_mat = np.hstack((onehot_test, demographics_test))  #merge the demographics info
y_pred = rf.predict(test_mat)
r2_RF_1, rmse =  evaluation(y_test, y_pred)
print('testing on filter samples...')

#word2vec encoder 单尺度编码
print(f,'RandomForestRegressor with embedding encoding...')
train_mat = embedding_encoder(X1_train, index_embedding)
test_mat = embedding_encoder(X1_test, index_embedding)
train_mat = np.hstack((train_mat, demographics_train))#merge the demographics info

rf = ensemble.RandomForestRegressor(n_estimators=10,random_state=90)
rf.fit(train_mat, y_train.ravel())                 #ravel()方法将数组维度拉成一维数组
test_mat = np.hstack((test_mat, demographics_test))#merge the demographics info
y_pred = rf.predict(test_mat)
r2_RF_2, rmse = evaluation(y_test, y_pred)

RandomForestRegressor with one_hot encoding...
开始步骤53/3:开始独热编码one_hot encoding...
开始步骤53/3:开始独热编码one_hot encoding...
y_test:(1569, 1)
y_pred:(1569,)
float64
MSE:0.0703171263228731, RMSE:0.2651737662795343, MAE:0.10516539739453161, R2:0.9217197944208495
testing on filter samples...
/content/gdrive/MyDrive/15684_new_2.csv RandomForestRegressor with embedding encoding...
开始步骤51/3:单尺度编码
开始步骤51/3:单尺度编码
y_test:(1569, 1)
y_pred:(1569,)
float64
MSE:0.1330823894033981, RMSE:0.36480459071042143, MAE:0.17538567136396185, R2:0.8518466645859811


### **2.3.2 多尺度随机森林模型**

In [19]:
#multichannel word2vec embedding encoder
print(f,'RandomForestRegressor with multichannel embedding encoding...')
train_mat = multichannel_embedding_encoder(X1_train_1, index_embedding, index_embedding1, index_embedding2, index_embedding3)
val_mat= multichannel_embedding_encoder(X1_val_1, index_embedding, index_embedding1, index_embedding2, index_embedding3)
test_mat = multichannel_embedding_encoder(X1_test, index_embedding, index_embedding1, index_embedding2, index_embedding3)

train_mat = np.hstack((train_mat, demographics_train_1))#训练集
val_mat = np.hstack((val_mat, demographics_val_1))#验证集
test_mat = np.hstack((test_mat, demographics_test))#测试集

rf = ensemble.RandomForestRegressor(n_estimators=10) #n_jobs=-1便是使用全部的CPU
rf.fit(train_mat, y_train_1.ravel())
#验证集预测 
y_RF_pred= rf.predict(val_mat)
R2_RF_3, rmse = evaluation(y_val_1, y_RF_pred) #验证集的拟合优度
#测试集预测 
train_mat=multichannel_embedding_encoder(X1_train, index_embedding, index_embedding1, index_embedding2, index_embedding3)
train_all = np.hstack((train_mat, demographics_train))#训练集+验证集
rf.fit(train_all, y_train.ravel()) 
y_RF_pred_3= rf.predict(test_mat) #测试集的预测值
R2_RF, rmse = evaluation(y_test, y_RF_pred_3)

/content/gdrive/MyDrive/15684_new_2.csv RandomForestRegressor with multichannel embedding encoding...
开始步骤52/3:开始多尺度编码...
开始步骤52/3:开始多尺度编码...
开始步骤52/3:开始多尺度编码...
y_test:(1412, 1)
y_pred:(1412,)
float64
MSE:0.0681524394138873, RMSE:0.26106022181459837, MAE:0.10606485127333316, R2:0.9254703754580028
开始步骤52/3:开始多尺度编码...
y_test:(1569, 1)
y_pred:(1569,)
float64
MSE:0.07210274586443305, RMSE:0.26851954466003597, MAE:0.10451736832364035, R2:0.919731961980746


In [28]:
#可视化
font2 = {'family': 'Times New Roman',
     'weight': 'normal',
     'size': 22,
     }
plt.figure(figsize=(15,5))
plt.plot(y_test,'rs',label='true value')
plt.plot(y_RF_pred_3,'go',label='predict value')
plt.title('Random Forest_'+'modelscore: %f'%R2_RF,font2)
plt.xticks(size=22)
plt.yticks(size=22)
plt.xlabel('Time Series',font2)
plt.ylabel('Medical Cost',font2)
plt.legend(loc=1,fontsize=14,frameon=True)
plt.savefig('/content/gdrive/MyDrive/RF_0.9197.png',dpi=500,bbox_inches = 'tight')
plt.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  
findfont: Font family ['Times New Roman'] not found. Falling back to DejaVu Sans.


# **3 新方法：Stacking**

In [ ]:
#不要运行
X_train, X_test, X1_train, X1_test, y_train, y_test, demographics_train, demographics_test = train_test_split(comm_X, comm_X1,comm_y, comm_demographics, test_size=0.1, random_state=seed)

In [ ]:
#不要运行
X1_train_1,X1_val_1, demographics_train_1,demographics_val_1,y_train_1,y_val_1=train_test_split( X1_train, demographics_train, y_train, test_size=0.1, random_state=90) 

## **3.1 blending**

In [ ]:
train_mat = multichannel_embedding_encoder(X1_train_1, index_embedding, index_embedding1, index_embedding2, index_embedding3)
val_mat= multichannel_embedding_encoder(X1_val_1, index_embedding, index_embedding1, index_embedding2, index_embedding3)
test_mat = multichannel_embedding_encoder(X1_test, index_embedding, index_embedding1, index_embedding2, index_embedding3)

train_mat = np.hstack((train_mat, demographics_train_1))#训练集
val_mat = np.hstack((val_mat, demographics_val_1))#验证集
test_mat = np.hstack((test_mat, demographics_test))#测试集

#测试集预测 
train_mat_all=multichannel_embedding_encoder(X1_train, index_embedding, index_embedding1, index_embedding2, index_embedding3)
train_all = np.hstack((train_mat_all, demographics_train))#训练集+验证集

开始多尺度编码...
开始多尺度编码...
开始多尺度编码...
开始多尺度编码...


In [ ]:
from sklearn import tree
from sklearn import linear_model
from sklearn import svm
from sklearn import neighbors
from sklearn import ensemble
from sklearn.ensemble import BaggingRegressor
#from sklearn.tree import ExtraTreeRegressor
import xgboost as xgb
#模型融合中使用到的各个单模型
clfs = [ensemble.AdaBoostRegressor(random_state=90),
        tree.DecisionTreeRegressor(random_state=90),
        tree.ExtraTreeRegressor(random_state=90),
        ensemble.GradientBoostingRegressor(random_state=90)]

dataset_d1 = np.zeros((val_mat.shape[0], len(clfs)))
dataset_d2 = np.zeros((X1_test.shape[0], len(clfs)))
 
for j, clf in enumerate(clfs):
    
    clf.fit(train_mat, y_train_1) #依次训练各个单模型
    y_submission = clf.predict(val_mat)[:, 1]
    dataset_d1[:, j] = y_submission
    dataset_d2[:, j] = clf.predict(X1_test)[:, 1]  #对于测试集，直接用这k个模型的预测值作为新的特征。
    #print("val auc Score: %f" % roc_auc_score(y_predict, dataset_d2[:, j]))

#融合使用的模型
clf = ensemble.RandomForestRegressor(n_estimators=500, n_jobs=-1)
clf.fit(dataset_d1, y_val_1)
R2=r2_score(y_test, y_submission)
print(R2)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


IndexError: ignored

## **3.2 Stacking**

In [ ]:
!pip install mlxtend 

### **3.2.1 尝试1**

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import itertools
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from sklearn import datasets

from sklearn import tree
from sklearn import linear_model
from sklearn import svm
from sklearn import neighbors
from sklearn import ensemble
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import ExtraTreeRegressor
import xgboost as xgb

#from sklearn.linear_model import LogisticRegression
#from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestRegressor
#from mlxtend.classifier import StackingClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from mlxtend.plotting import plot_learning_curves
from mlxtend.plotting import plot_decision_regions

X =train_all
y=y_train

clf1 = ensemble.GradientBoostingRegressor(random_state=90) 
clf2 = ensemble.RandomForestRegressor(random_state=90)
clf3 = ensemble.AdaBoostRegressor(random_state=90)
#lr = BaggingRegressor(n_estimators=10,random_state=90)
#sclf = StackingRegressor(Regressors=[clf1, clf2, clf3], meta_Regressor=lr)


label = ['GradientBoosting', 'Random Forest', 'AdaBoost', 'Stacking Regressor']
clf_list = [clf1, clf2, clf3]
    
fig = plt.figure(figsize=(10,8))
gs = gridspec.GridSpec(2, 2)
grid = itertools.product([0,1],repeat=2)


clf_cv_mean = []
clf_cv_std = []
for clf, label, grd in zip(clf_list, label, grid):
        
    scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
    print("Accuracy: %.2f (+/- %.2f) [%s]" %(scores.mean(), scores.std(), label))
    clf_cv_mean.append(scores.mean())
    clf_cv_std.append(scores.std())
        
    clf.fit(X, y)
    ax = plt.subplot(gs[grd[0], grd[1]])
    fig = plot_decision_regions(X=X, y=y, clf=clf)
    plt.title(label)
    vclf=clf.predict(X1_test)
    R2=r2_score(y_test,vclf)
    print(R2)
 
plt.show()

### **3.2.2 尝试2**

In [ ]:
X_all = np.hstack((comm_X1, comm_demographics))#训练集+验证集
y=comm_y

In [ ]:
# coding=utf8
 
 
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import StratifiedKFold
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.datasets.samples_generator import make_blobs #聚类数据生成器其参数设置详见：https://blog.csdn.net/kevinelstri/article/details/52622960
 
'''创建训练的数据集'''
data, target = X_all,y
 
'''模型融合中使用到的各个单模型'''
clfs = [ensemble.GradientBoostingRegressor(random_state=90) ,
        ensemble.RandomForestRegressor(random_state=90),
        ensemble.AdaBoostRegressor(random_state=90),
        BaggingRegressor(n_estimators=10,random_state=90)]
 
'''切分一部分数据作为测试集'''
X, X_predict, y, y_predict = train_test_split(data, target, test_size=0.1, random_state=90)
 
 
dataset_blend_train = np.zeros((X.shape[0], len(clfs)))
dataset_blend_test = np.zeros((X_predict.shape[0], len(clfs)))
 
'''5折stacking'''
n_folds = 5
skf = list(StratifiedKFold(y, n_folds))
for j, clf in enumerate(clfs):
    '''依次训练各个单模型'''
    # print(j, clf)
    dataset_blend_test_j = np.zeros((X_predict.shape[0], len(skf)))
    for i, (train, test) in enumerate(skf):
        '''使用第i个部分作为预测，剩余的部分来训练模型，获得其预测的输出作为第i部分的新特征。'''
        # print("Fold", i)
        X_train, y_train, X_test, y_test = X[train], y[train], X[test], y[test]
        clf.fit(X_train, y_train)
        y_submission = clf.predict_proba(X_test)[:, 1]
        dataset_blend_train[test, j] = y_submission
        dataset_blend_test_j[:, i] = clf.predict_proba(X_predict)[:, 1]
    '''对于测试集，直接用这k个模型的预测值均值作为新的特征。'''
    dataset_blend_test[:, j] = dataset_blend_test_j.mean(1)
    print("val auc Score: %f" % roc_auc_score(y_predict, dataset_blend_test[:, j]))
# clf = LogisticRegression()
clf = BaggingRegressor(n_estimators=10,random_state=90)
clf.fit(dataset_blend_train, y)
y_submission = clf.predict(dataset_blend_test)[:, 1]
 
print("Linear stretch of predictions to [0,1]")
#y_submission = (y_submission - y_submission.min()) / (y_submission.max() - y_submission.min())
print("blend result")
print("val r2 Score: %f" % (r2_score(y_predict, y_submission)))



### **3.2.3 尝试3**

In [ ]:
from sklearn.model_selection import train_test_split, ShuffleSplit, KFold
from sklearn.model_selection import cross_val_score, cross_validate, ShuffleSplit, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.svm import SVR
import matplotlib.pyplot as plt


# Loading Train & Test dataset
#df_train = pd.read_csv('001_train.csv')
#df_test = pd.read_csv('001_test.csv')


# Predicators & Target columns
#predicators = ['Item_Weight', 'Item_Fat_Content',
       'Item_MRP', 'Item_Visibility', 'Items_ID',
        'Outlet_Size','Outlet_Age', 'Outlet_Location_Type', 'Outlet_Type' ]
#target = 'Item_Outlet_Sales'

# Choosing base models
base_models = [DecisionTreeRegressor, RandomForestRegressor, BaggingRegressor, AdaBoostRegressor, XGBRegressor]

# Setting up train dataset for ensemble model
row_train = len(train_all) #数据量（长度）
col_train = len(base_models) #所用模型数量
stacking_train_dataset1 = np.zeros(shape = (row_train, col_train))
stacking_train_dataset = pd.DataFrame(data= stacking_train_dataset1)
# Setting up test dataset for ensemble model
row_test = len(X1_test)
col_test = len(base_models)
stacking_test_dataset1 = np.zeros(shape = (row_test, col_test))
stacking_test_dataset = pd.DataFrame(data = stacking_test_dataset1)


# Setting up KFold splitter
number_of_split = 10
split = KFold(n_splits = number_of_split, random_state = 42, shuffle = True)

#Training Model and Filling up Train & Test dataset for stacking ensemble
for i, base_alg in enumerate(base_models):

    counter = randint(0, number_of_split)
    inner_counter = 0

    for trainix , testix in split.split(stacking_train_dataset):
        if inner_counter == counter:
            x_train = train_all
            y_train = y_train 

            model = base_alg()
            print('training : %s'%(type(model)))
            model.fit(x_train, y_train)

            stacking_train_dataset[i] = model.predict(X1_val_1)
            stacking_test_dataset[i] = model.predict(df_test[predicators])

        inner_counter += 1


# You can change the ensemble model here
ensemble_model = LinearRegression()

# Ensemble Model Prediction
stacking_train_dataset['Ensemble'] = ensemble_model.fit(stacking_train_dataset, df_train[target]).predict(stacking_train_dataset)

# Preparing for sumbission
IDcol = ['Item_Identifier', 'Outlet_Identifier']
submission = df_test[IDcol]
submission[target] = ensemble_model.predict(stacking_test_dataset)
submission.to_csv('Submit01.csv', index = False)

stacking_test_dataset['Ensemble_Output'] = ensemble_model.predict(stacking_test_dataset)

# Saving Created Datasets
stacking_test_dataset.to_csv('stacking_test.csv', index = False)
stacking_train_dataset.to_csv('stacking_train.csv', index = False)





# **4、组合预测.py**

## **4.1 预测1（特征）**

### **4.1.1 数据整理**

In [ ]:
y_CNN_pred

array([[5.7471347],
       [5.737805 ],
       [5.7776055],
       ...,
       [5.7335706],
       [5.7313743],
       [5.750322 ]], dtype=float32)

In [ ]:
y_RF_pred_1.shape

NameError: ignored

In [ ]:
y_RF_pred_new= np.expand_dims(y_RF_pred_1,axis=1)
y_RF_pred_new.shape

(1569, 1)

In [ ]:
y_RF_pred_new

array([[5.50467526],
       [5.70948706],
       [5.90536427],
       ...,
       [5.72725911],
       [5.73694811],
       [5.67266374]])

### **4.1.2 数据预测**

In [ ]:
print(r2)
print(R2_RF_1)

0.8291481184004648
0.9051198378220215


In [ ]:
t=r2+R2_RF_1
Y_predict = (y_CNN_pred *r2+y_RF_pred_new* R2_RF_1)/t
print(Y_predict)
print(Y_predict.shape)

[[5.87565263]
 [5.62744704]
 [5.8227331 ]
 ...
 [5.64986356]
 [5.62981377]
 [5.62561448]]
(1569, 1)


In [ ]:
score = r2_score(y_test, Y_predict)
print(score)

0.8835185924298106


## **4.2 预测2（特征+疾病）**

### **4.2.1 数据整理**

In [44]:
y_RF_pred_new= np.expand_dims(y_RF_pred_2,axis=1)
y_RF_pred_new.shape

NameError: ignored

### **4.2.2 数据预测**

In [ ]:
print(r2)
print(R2_RF_2)

0.8291481184004648
0.9391008234392352


In [ ]:
t=r2+R2_RF_2
Y_predict = (y_CNN_pred *r2 + y_RF_pred_new* R2_RF_2)/t
print(Y_predict)
print(Y_predict.shape)

[[6.11147823]
 [5.62597148]
 [5.7174304 ]
 ...
 [5.64337355]
 [5.61457943]
 [5.66451593]]
(1569, 1)


In [ ]:
score = r2_score(y_test, Y_predict)
print(score)

0.9037265637633737


In [ ]:
def Weighted_method(y_CNN_pred, y_RF_pred_new , w=[1/3, 1/3]):
  t=w[0]+w[1]
  Weighted_result = (y_CNN_pred *w[0] + y_RF_pred_new*w[1])/t 
  return Weighted_result
w=[]
w.append(r2)
w.append(R2_RF_2)
Weighted_pre = Weighted_method(y_CNN_pred, y_RF_pred_new ,w) 
print('Weighted_pre MAE:',mean_absolute_error(y_test, Weighted_pre))   # 会发现这个效果会提高一些
print('Weighted_pre r2:',r2_score(y_test, Weighted_pre))   # 会发现这个效果会提高一些

## **4.3 RF3组合预测**

### **4.3.1 拟合优度权重**

In [ ]:
y_RF_pred_new= np.expand_dims(y_RF_pred_3,axis=1)
y_RF_pred_new.shape

(1569, 1)

In [ ]:
print(r2)
print(R2_RF_3)

0.9370573616906956
0.9416302371979749


In [ ]:

t=r2+R2_RF_3
Y_predict = (y_CNN_pred *(r2) + y_RF_pred_new* (R2_RF_3))/t
print(Y_predict)
print(Y_predict.shape)

score = r2_score(y_test, Y_predict)
print(score)

[[5.87321832]
 [5.72073101]
 [5.74058307]
 ...
 [5.71861902]
 [5.71752356]
 [5.76541073]]
(1569, 1)
0.922890864492953


In [ ]:
r2=0.904542947043012
t=r2+R2_RF_3
Y_predict = (y_CNN_pred *(r2) + y_RF_pred_new* (R2_RF_3))/t
print(Y_predict)
print(Y_predict.shape)

score = r2_score(y_test, Y_predict)
print(score)

[[5.90304553]
 [5.71632628]
 [5.85498241]
 ...
 [5.7052852 ]
 [5.69435484]
 [5.72257327]]
(1569, 1)
0.9173941478197482


### **4.3.2 以 1/MAE 为权重**

In [22]:
modelpath = '有时间间隔_5月16日(下午)_'+'第7次训练_'+'MG_merge' + path # %i
#定义模型
model= ATTENTION_multi_channel_split_model(demgras_dim)
test_ATTENTION_multi_channel_split_model(model, modelpath, X1_test, demographics_test,y_test,3)
model.load_weights("/content/gdrive/MyDrive/cnn_model/%s.hdf5"%(modelpath))
y_CNN_pred = model.predict([X1_test, demographics_test], batch_size=args.batchsize, verbose=1)
r22,rmse = evaluation(y_test, y_CNN_pred)
print("r2:",r22,"rmse:",rmse)
r2_CNN_test.append(r22)

demgras_dim:10
demgras_in.shape:(None, 10)
通道数:1300
sigmoid激活，权重归一化: KerasTensor(type_spec=TensorSpec(shape=(1, 1, 1, 1300), dtype=tf.float32, name=None), name='tf.math.sigmoid_23/Sigmoid:0', description="created by layer 'tf.math.sigmoid_23'")
(1, 1, None, 1300)
Model: "model_133"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_47 (InputLayer)          [(None, 17)]         0           []                               
                                                                                                  
 embedding_92 (Embedding)       (None, 17, 900)      649800      ['input_47[0][0]']               
                                                                                                  
 embedding_93 (Embedding)       (None, 17, 900)      649800      ['input_47[0][0]']               
                        

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


                                                                  'global_max_pooling1d_278[0][0]'
                                                                 , 'global_max_pooling1d_279[0][0]
                                                                 ']                               
                                                                                                  
 add_70 (Add)                   (None, 100)          0           ['global_max_pooling1d_280[0][0]'
                                                                 , 'global_max_pooling1d_281[0][0]
                                                                 ',                               
                                                                  'global_max_pooling1d_282[0][0]'
                                                                 , 'global_max_pooling1d_283[0][0]
                                                                 ']                               
          

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:446: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:457: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:468: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warnin

开始步骤92/2:预测可视化-医疗费用


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:457: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:468: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:181: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warnin

!!开始画图：预测值与真实值
y_test: (1569, 1)
y_pred: (1569, 1)
y_pred.ndim: 2
y_test.ndim: 2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:195: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:206: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


13/13 [==============================] - 0s 18ms/step
y_test:(1569, 1)
y_pred:(1569, 1)
float32
MSE:0.07077091932296753, RMSE:0.2660280466079712, MAE:0.10452725738286972, R2:0.9212146105790974
r2: 0.9212146105790974 rmse: 0.26602805


In [23]:
y_RF_pred_new= np.expand_dims(y_RF_pred_3,axis=1)
y_RF_pred_new.shape
a=1/0.104527
b=1/0.104517
t=a+b
Y_predict = (y_CNN_pred *(a) + y_RF_pred_new* (b))/t
print(Y_predict)
print(Y_predict.shape)

score = r2_score(y_test, Y_predict)
print(score)

[[8.14885529]
 [8.1756007 ]
 [5.75853155]
 ...
 [8.25161563]
 [5.79005329]
 [5.70253697]]
(1569, 1)
0.9270155410507026


In [24]:
R2_组合, rmse = evaluation(y_test, Y_predict)

y_test:(1569, 1)
y_pred:(1569, 1)
float64
MSE:0.06556009123347507, RMSE:0.25604704886695157, MAE:0.10117292599279153, R2:0.9270155410507026


### **没有时间间隔**

In [25]:
modelpath = '没有时间间隔_5月16日(下午)_'+'第2次训练_'+'MG_merge' + path # %i
#定义模型
model= ATTENTION_multi_channel_split_model(demgras_dim)
test_ATTENTION_multi_channel_split_model(model, modelpath, X1_test, demographics_test,y_test,3)
model.load_weights("/content/gdrive/MyDrive/cnn_model/%s.hdf5"%(modelpath))
y_CNN_pred = model.predict([X1_test, demographics_test], batch_size=args.batchsize, verbose=1)
r22,rmse = evaluation(y_test, y_CNN_pred)
print("r2:",r22,"rmse:",rmse)
r2_CNN_test.append(r22)

demgras_dim:10
demgras_in.shape:(None, 10)
通道数:1300
sigmoid激活，权重归一化: KerasTensor(type_spec=TensorSpec(shape=(1, 1, 1, 1300), dtype=tf.float32, name=None), name='tf.math.sigmoid_24/Sigmoid:0', description="created by layer 'tf.math.sigmoid_24'")
(1, 1, None, 1300)
Model: "model_134"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_49 (InputLayer)          [(None, 17)]         0           []                               
                                                                                                  
 embedding_96 (Embedding)       (None, 17, 900)      649800      ['input_49[0][0]']               
                                                                                                  
 embedding_97 (Embedding)       (None, 17, 900)      649800      ['input_49[0][0]']               
                        

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


 lobalAveragePooling1D)                                                                           
                                                                                                  
 reshape_24 (Reshape)           (1, 1, 1, 1300)      0           ['global_average_pooling1d_24[0][
                                                                 0]']                             
                                                                                                  
 dense_97 (Dense)               (1, 1, 1, 650)       845650      ['reshape_24[0][0]']             
                                                                                                  
 tf.nn.relu_24 (TFOpLambda)     (1, 1, 1, 650)       0           ['dense_97[0][0]']               
                                                                                                  
 dense_98 (Dense)               (1, 1, 1, 1300)      846300      ['tf.nn.relu_24[0][0]']          
          

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:446: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:457: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:468: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warnin

开始步骤92/2:预测可视化-医疗费用


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:457: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:468: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:181: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warnin

!!开始画图：预测值与真实值
y_test: (1569, 1)
y_pred: (1569, 1)
y_pred.ndim: 2
y_test.ndim: 2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:195: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:206: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


13/13 [==============================] - 0s 17ms/step
y_test:(1569, 1)
y_pred:(1569, 1)
float32
MSE:0.07188301533460617, RMSE:0.268110066652298, MAE:0.1207733228802681, R2:0.9199765740763938
r2: 0.9199765740763938 rmse: 0.26811007


In [26]:
y_RF_pred_new= np.expand_dims(y_RF_pred_3,axis=1)
y_RF_pred_new.shape
a=1/0.120773
b=1/0.104517
t=a+b
Y_predict = (y_CNN_pred *(a) + y_RF_pred_new* (b))/t
print(Y_predict)
print(Y_predict.shape)

score = r2_score(y_test, Y_predict)
print(score)

[[8.16601276]
 [8.20135302]
 [5.77544829]
 ...
 [8.28682786]
 [5.80673686]
 [5.71162716]]
(1569, 1)
0.9257909201932832


In [27]:
R2_组合, rmse = evaluation(y_test, Y_predict)

y_test:(1569, 1)
y_pred:(1569, 1)
float64
MSE:0.06666013713769428, RMSE:0.25818624505905474, MAE:0.10891758684677759, R2:0.9257909201932832


### **4.3.3 其他权重确定方法**

In [ ]:
y_CNN_hat

array([[5.732129],
       [5.752484],
       [5.740054],
       ...,
       [8.680058],
       [5.695688],
       [5.757047]], dtype=float32)

In [ ]:
y_CNN_pred

array([[5.7357826],
       [5.7293897],
       [6.011958 ],
       ...,
       [5.706889 ],
       [5.6846137],
       [5.6890583]], dtype=float32)

In [ ]:
y_CNN_hat_1=pd.DataFrame(y_CNN_hat)
y_CNN_hat_1.to_csv('/content/gdrive/MyDrive/y_CNN_hat.csv')
y_CNN_pred_1=pd.DataFrame(y_CNN_pred)
y_CNN_pred_1.to_csv('/content/gdrive/MyDrive/y_CNN_pred.csv')

### **4.3.4 遗传算法**

In [ ]:
#coding=utf-8
import warnings
warnings.filterwarnings("ignore")

import numpy as np
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import random
import math
from sklearn import metrics
from sklearn.model_selection import train_test_split
 
 
generations = 20   # 繁殖代数 100
pop_size = 20      # 种群数量  500
max_value = 10      # 基因中允许出现的最大值  
chrom_length = 8    # 染色体长度  
pc = 0.6            # 交配概率  
pm = 0.01           # 变异概率  
results = [[]]      # 存储每一代的最优解，N个三元组（auc最高值, n_estimators, max_depth）  
fit_value = []      # 个体适应度  
fit_mean = []       # 平均适应度 
pop = [[0, 1, 0, 1, 0, 1, 0, 1] for i in range(pop_size)] # 初始化种群中所有个体的基因初始序列

#可行解在遗传算法中均被称为染色体 
 
 
 
'''
n_estimators 取 {10、20、30、40、50、60、70、80、90、100、110、120、130、140、150、160}
max_depth 取 {1、2、3、4、5、6、7、8、9、10、11、12、13、14、15、16} 
（1111，1111）基因组8位长
'''
def aimFunction(a,b):
  
    rf = y_val_1 - a*y_train_1 - b*y_RF_pred
   
    return rf
 
# Step 1 : 对参数进行编码（用于初始化基因序列，可以选择初始化基因序列，本函数省略）
def geneEncoding(pop_size, chrom_length):  
    pop = [[]]
    for i in range(pop_size):
        temp = []
        for j in range(chrom_length):
            temp.append(random.randint(0, 1))
        pop.append(temp)
    return pop[1:]
 
 
# Step 2 : 计算个体的目标函数值
def cal_obj_value(pop):
    objvalue = []
    variable = decodechrom(pop)
    for i in range(len(variable)):
        tempVar = variable[i]
        n_estimators_value = (tempVar[0] + 1) * 10
        max_depth_value = tempVar[1] + 1
        aucValue = aimFunction(a,b)
        objvalue.append(aucValue)
    return objvalue #目标函数值objvalue[m] 与个体基因 pop[m] 对应 
 
 
 
 
# 对每个个体进行解码，并拆分成单个变量，返回 n_estimators 和 max_depth
def decodechrom(pop):
    variable = []
    n_estimators_value = []
    max_depth_value = []
    for i in range(len(pop)):
        res = []
        
        # 计算第一个变量值，即 0101->10(逆转)
        temp1 = pop[i][0:4]
        preValue = 0
        for pre in range(4):
            preValue += temp1[pre] * (math.pow(2, pre))
        res.append(int(preValue))
        
        # 计算第二个变量值
        temp2 = pop[i][4:8]
        aftValue = 0
        for aft in range(4):
            aftValue += temp2[aft] * (math.pow(2, aft))
        res.append(int(aftValue))
        variable.append(res)
    return variable
 
 
 
 
# Step 3: 计算个体的适应值（计算最大值，于是就淘汰负值就好了）
def calfitvalue(obj_value):
    fit_value = []
    temp = 0.0
    Cmin = 0
    for i in range(len(obj_value)):
        if(obj_value[i] + Cmin > 0):
            temp = Cmin + obj_value[i]
        else:
            temp = 0.0
        fit_value.append(temp)
    return fit_value
 
 
 
 
# Step 4: 找出适应函数值中最大值，和对应的个体
def best(pop, fit_value):
    best_individual = pop[0]
    best_fit = fit_value[0]
    for i in range(1, len(pop)):
        if(fit_value[i] > best_fit):
            best_fit = fit_value[i]
            best_individual = pop[i]
    return [best_individual, best_fit]
 
 
 
 
# Step 5: 每次繁殖，将最好的结果记录下来(将二进制转化为十进制)
def b2d(best_individual):
    temp1 = best_individual[0:4]
    preValue = 0
    for pre in range(4):
        preValue += temp1[pre] * (math.pow(2, pre))
    preValue = preValue + 1
    preValue = preValue * 10
    
    # 计算第二个变量值
    temp2 = best_individual[4:8]
    aftValue = 0
    for aft in range(4):
        aftValue += temp2[aft] * (math.pow(2, aft))
    aftValue = aftValue + 1
    return int(preValue), int(aftValue)
 
 
 
 
# Step 6: 自然选择（轮盘赌算法）
def selection(pop, fit_value):
    # 计算每个适应值的概率
    new_fit_value = []
    total_fit = sum(fit_value)
    for i in range(len(fit_value)):
        new_fit_value.append(fit_value[i] / total_fit)
    # 计算每个适应值的累积概率
    cumsum(new_fit_value)
    # 生成随机浮点数序列
    ms = []
    pop_len = len(pop)
    for i in range(pop_len):
        ms.append(random.random())
    # 对生成的随机浮点数序列进行排序
    ms.sort()
    # 轮盘赌算法（选中的个体成为下一轮，没有被选中的直接淘汰，被选中的个体代替）
    fitin = 0
    newin = 0
    newpop = pop
    while newin < pop_len:
        if(ms[newin] < new_fit_value[fitin]):
            newpop[newin] = pop[fitin]
            newin = newin + 1
        else:
            fitin = fitin + 1
    pop = newpop
 
 
# 求适应值的总和
def sum(fit_value):
    total = 0
    for i in range(len(fit_value)):
        total += fit_value[i]
    return total
 
 
# 计算累积概率
def cumsum(fit_value):
    temp=[]
    for i in range(len(fit_value)):
        t = 0
        j = 0
        while(j <= i):
            t += fit_value[j]
            j = j + 1
        temp.append(t)
    for i in range(len(fit_value)):
        fit_value[i]=temp[i]
 
 
# Step 7: 交叉繁殖
def crossover(pop, pc): #个体间交叉，实现基因交换
    poplen = len(pop)
    for i in range(poplen - 1):
        if(random.random() < pc):
            cpoint = random.randint(0,len(pop[0]))
            temp1 = []
            temp2 = []
            temp1.extend(pop[i][0 : cpoint])
            temp1.extend(pop[i+1][cpoint : len(pop[i])])
            temp2.extend(pop[i+1][0 : cpoint])
            temp2.extend(pop[i][cpoint : len(pop[i])])
            pop[i] = temp1
            pop[i+1] = temp2
 
 
 
 
# Step 8: 基因突变
def mutation(pop, pm):
    px = len(pop)
    py = len(pop[0])
    for i in range(px):
        if(random.random() < pm):
            mpoint = random.randint(0,py-1)
            if(pop[i][mpoint] == 1):
                pop[i][mpoint] = 0
            else:
                pop[i][mpoint] = 1
 
 
 
 
if __name__ == '__main__':
    # pop = geneEncoding(pop_size, chrom_length)
    for i in range(generations):
        print("第 " + str(i) + " 代开始繁殖......")
        obj_value = cal_obj_value(pop) # 计算目标函数值
        #print(obj_value)
        fit_value = calfitvalue(obj_value) #计算个体的适应值
        # print(fit_value)
        [best_individual, best_fit] = best(pop, fit_value) #选出最好的个体和最好的函数值
        # print("best_individual: "+ str(best_individual))
        temp_n_estimator, temp_max_depth = b2d(best_individual)
        results.append([best_fit, temp_n_estimator, temp_max_depth]) #每次繁殖，将最好的结果记录下来
        print(str(best_individual) + " " + str(best_fit))
        selection(pop, fit_value) #自然选择，淘汰掉一部分适应性低的个体
        crossover(pop, pc) #交叉繁殖
        mutation(pop, pc) #基因突变
    # print(results)
    results.sort()
    print(results[-1])